In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\ajl0618\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import pyodbc
import numpy as np
from datetime import date

In [3]:
spark = SparkSession.builder.appName("PBPPlanBuilder").getOrCreate()

In [4]:
#system parameters\
PBP_SOURCE_FOLDER = 'PBP_Benefits_2024/'
DR_TARGET_FOLDER = 'PBP_Benefits_2024_Results/'

In [5]:
# year parameter for CMS

PLAN_YEAR = 2024
RX_CATASTROHPIC_LIMIT = 8000
RX_INITIAL_COVERAGE_LIMIT = 5030
DEDAULT_RX_DEDUCTIBLE = 545
MEDICARE_DEDUCTIBLE_PART_A = 1632
MEDICARE_DEDUCTIBLE_PART_B = 240

In [6]:
# load all required files
def load_csv(csv_file_path):
    return spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)

# drop views from memory if the view exist
for t in spark.catalog.listTables():
    spark.catalog.dropTempView(t.name)

df_pbp_section_A = load_csv(PBP_SOURCE_FOLDER + 'pbp_Section_A.txt')

df_pbp_section_A.createTempView('pbp_section_A')

df_pbp_mrx = load_csv(PBP_SOURCE_FOLDER + 'pbp_mrx.txt')
df_pbp_mrx.createTempView('pbp_mrx')

df_pbp_Section_C = load_csv(PBP_SOURCE_FOLDER + 'pbp_Section_C.txt')
df_pbp_Section_C.createTempView('pbp_Section_C')

df_pbp_Section_C_OON = load_csv(PBP_SOURCE_FOLDER + 'pbp_Section_C_OON.txt')
df_pbp_Section_C_OON.createTempView('pbp_Section_C_OON')

df_pbp_Section_C_POS = load_csv(PBP_SOURCE_FOLDER + 'pbp_Section_C_POS.txt')
df_pbp_Section_C_POS.createTempView('pbp_Section_C_POS')

df_pbp_Section_D = load_csv(PBP_SOURCE_FOLDER + 'pbp_Section_D.txt')
df_pbp_Section_D.createTempView('pbp_Section_D')

df_pbp_b1a_inpat_hosp = load_csv(PBP_SOURCE_FOLDER + 'pbp_b1a_inpat_hosp.txt')
df_pbp_b1a_inpat_hosp.createTempView('pbp_b1a_inpat_hosp')

df_pbp_b2_snf = load_csv(PBP_SOURCE_FOLDER + 'pbp_b2_snf.txt')
df_pbp_b2_snf.createTempView('pbp_b2_snf')

df_pbp_step2 = load_csv(PBP_SOURCE_FOLDER + 'pbp_step2.txt')
df_pbp_step2.createTempView('pbp_step2')

df_pbp_b4_emerg_urgent =load_csv(PBP_SOURCE_FOLDER + 'pbp_b4_emerg_urgent.txt')
df_pbp_b4_emerg_urgent.createTempView('pbp_b4_emerg_urgent')

df_pbp_b7_health_prof = load_csv(PBP_SOURCE_FOLDER + 'pbp_b7_health_prof.txt')
df_pbp_b7_health_prof.createTempView('pbp_b7_health_prof')

df_pbp_b8_clin_diag_ther =load_csv(PBP_SOURCE_FOLDER + 'pbp_b8_clin_diag_ther.txt')
df_pbp_b8_clin_diag_ther.createTempView('pbp_b8_clin_diag_ther')

df_pbp_b9_outpat_hosp = load_csv(PBP_SOURCE_FOLDER + 'pbp_b9_outpat_hosp.txt')
df_pbp_b9_outpat_hosp.createTempView('pbp_b9_outpat_hosp')

df_pbp_b10_amb_trans =load_csv(PBP_SOURCE_FOLDER + 'pbp_b10_amb_trans.txt')
df_pbp_b10_amb_trans.createTempView('pbp_b10_amb_trans')

df_pbp_b11_dme_prosth_orth_sup =load_csv(PBP_SOURCE_FOLDER + 'pbp_b11_dme_prosth_orth_sup.txt')
df_pbp_b11_dme_prosth_orth_sup.createTempView('pbp_b11_dme_prosth_orth_sup')

df_pbp_b13_other_services =load_csv(PBP_SOURCE_FOLDER + 'pbp_b13_other_services.txt')
df_pbp_b13_other_services.createTempView('pbp_b13_other_services')

df_pbp_b13_b19b_other_services_vbid_uf =load_csv(PBP_SOURCE_FOLDER + 'pbp_b13_b19b_other_services_vbid_uf.txt')
df_pbp_b13_b19b_other_services_vbid_uf.createTempView('pbp_b13_b19b_other_services_vbid_uf')

df_pbp_b14_preventive =load_csv(PBP_SOURCE_FOLDER + 'pbp_b14_preventive.txt')
df_pbp_b14_preventive.createTempView('pbp_b14_preventive')

df_pbp_b15_partb_rx_drugs= load_csv(PBP_SOURCE_FOLDER + 'pbp_b15_partb_rx_drugs.txt')
df_pbp_b15_partb_rx_drugs.createTempView('pbp_b15_partb_rx_drugs')

df_pbp_b16_dental= load_csv(PBP_SOURCE_FOLDER + 'pbp_b16_dental.txt')
df_pbp_b16_dental.createTempView('pbp_b16_dental')

df_pbp_b17_eye_exams_wear_aids =load_csv(PBP_SOURCE_FOLDER + 'pbp_b17_eye_exams_wear.txt')
df_pbp_b17_eye_exams_wear_aids.createTempView('pbp_b17_eye_exams_wear')

df_pbp_b18_hearing_exams_aids =load_csv(PBP_SOURCE_FOLDER + 'pbp_b18_hearing_exams_aids.txt')
df_pbp_b18_hearing_exams_aids.createTempView('pbp_b18_hearing_exams_aids')

In [7]:
#List of utility functions
def write_to_csv_file(df, file_name):
    pandas_df = df.toPandas()
    pandas_df.to_csv(DR_TARGET_FOLDER + file_name + '.csv', index=False)

def read_pd_from_csv_file(file_name):
    return pd.read_csv(DR_TARGET_FOLDER + file_name + '.csv')

def write_pd_to_csv(df, file_name):
    df.to_csv(DR_TARGET_FOLDER + file_name + '.csv', index=False)

def convert_to_int(field, null_value):
    if field is None:
        return null_value
    return int(field)

def convert_to_currency(float_field):
    return '${:,.2f}'.format(float_field)

def convert_to_currency_no_decimal(float_field):
    return '${:,.0f}'.format(float_field)

def drop_pbp_mrx_columns(df):
	pbp_mrx_columns = []
	for column_name in df.columns:
		if column_name.lower().startswith('pbp_') or column_name.lower().startswith('mrx_'):
			pbp_mrx_columns.append(column_name)
	df = df.drop(pbp_mrx_columns, axis=1)
	return df


def drop_pbp_mrx_columns(df):
	pbp_mrx_columns = []
	for column_name in df.columns:
		if column_name.lower().startswith('pbp_') or column_name.lower().startswith('mrx_'):
			pbp_mrx_columns.append(column_name)
	df = df.drop(pbp_mrx_columns, axis=1)
	return df

def get_medicare_site_url(qid):
    contractid = qid[:5]
    planid = qid[5:8]
    segmentid = qid[8:]
    return f'https://www.medicare.gov/plan-compare/#/plan-details/{PLAN_YEAR}-{contractid}-{planid}-{int(segmentid)}?year={PLAN_YEAR}&lang=en#benefits'


In [8]:
# Carrier, Organization, Plans
# Source: HPMS..[usp_loadCarriersPUF] 

def get_medical_plan_type_code(pbp_a_plan_type):
	'''
	Source: hpms.dbo.f_getMedicalPlanTypeCode
	'''
	if pbp_a_plan_type is not None:
		pbp_a_plan_type = int(pbp_a_plan_type)
		if pbp_a_plan_type == 1:return f'HMO'
		if pbp_a_plan_type == 2: return 'HMOPOS'
		if pbp_a_plan_type == 4: return 'Local PPO'
		if pbp_a_plan_type == 5: return 'PSO (State License)'#??
		if pbp_a_plan_type == 7: return 'MSA'
		if pbp_a_plan_type == 8: return 'RFB PFFS'
		if pbp_a_plan_type == 9: return 'PFFS'
		if pbp_a_plan_type == 18: return '1876 Cost'
		if pbp_a_plan_type == 19: return 'HCPP - 1833 Cost'
		if pbp_a_plan_type == 20: return 'National Pace'
		if pbp_a_plan_type == 29: return 'Medicare Prescription Drug Plan'
		if pbp_a_plan_type == 30: return 'Employer/Union Only Direct Contract PDP'
		if pbp_a_plan_type == 31: return 'Regional PPO'
		if pbp_a_plan_type == 32: return 'Fallback'
		if pbp_a_plan_type == 40: return 'Employer/Union Only Direct Contract PFFS'
		if pbp_a_plan_type == 42: return 'RFB HMO'
		if pbp_a_plan_type == 43: return 'RFB HMOPOS'
		if pbp_a_plan_type == 44: return 'RFB Local PPO'
		if pbp_a_plan_type == 45: return 'RFB PSO (State License)'
		if pbp_a_plan_type == 47: return 'Employer Direct PPO'
		if pbp_a_plan_type == 48: return 'MMP HMO'
		if pbp_a_plan_type == 49: return 'MMP HMOPOS'
	return 'UNKNOWN'


query = f'''
SELECT 
	{PLAN_YEAR} as PlanYear,    
	PBP_A_ORG_MARKETING_NAME as CarrierName, pbp_a_org_name as OrganizationName, PBP_A_ORG_WEBSITE as WebSiteAddress,
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
	PBP_A_PLAN_NAME as PlanName, 
	PBP_A_PLAN_GEOG_NAME as GeoName, 
	m.PBP_A_PLAN_TYPE as MedicalPlanType,
	PBP_D_MPLUSC_PREMIUM as HealthPlanPremium, 
	MRX_ALT_COV_LMT_AMT as InitialCoverageLimit,
 	pbp_d_mco_pay_reduct_amt as PartBPremiumReduction,
	pbp_a_phys_web_addr as PhysicianSearchURL, 
 	pbp_a_curmbr_phone as MemberPhoneNumber, 
	pbp_a_prombr_phone as NonMemberPhoneNumber,
	pbp_a_ttytdd_curmbr_phone as TTYTDDPhoneNumber,
	pbp_a_eghp_yn, pbp_a_org_type, pbp_a_snp_pct, pbp_a_snp_cond, m.pbp_a_plan_type,
	pbp_a_special_need_flag, pbp_a_special_need_plan_type, 
	mrx_alt_ded_amount, mrx_alt_ded_charge, mrx_benefit_type
FROM pbp_section_A a 
	left join pbp_mrx m on a.bid_id = m.bid_id
	left join pbp_Section_D d on a.bid_id = d.bid_id
	where cast(pbp_a_eghp_yn as int) = 2 and cast(m.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
	and CAST(a.pbp_a_plan_identifier1 AS INT) < 800
'''

df_plans = spark.sql(query)
write_to_csv_file(df_plans, 'Plans')

#Add calculated columns

from PBP_Benefit_Text import  Plan # Logic implemented in Benefit Module

df_plans = read_pd_from_csv_file('Plans')
df_plans['PlanType'] = df_plans.apply(lambda x: Plan.get_PlanType(x), axis=1)
df_plans['SNPType'] =  df_plans.apply(lambda x: Plan.get_SNPType(x), axis=1)
df_plans['QID'] = df_plans.apply(lambda x: Plan.get_QID(x), axis=1)
df_plans['DrugDeductibleLimit'] = df_plans.apply(lambda x: Plan.get_DrugDeductibleLimit(x), axis=1)
df_plans['MedicalPlanType'] = df_plans['MedicalPlanType'].apply(lambda x: get_medical_plan_type_code(x))
df_plans = drop_pbp_mrx_columns(df_plans)

write_pd_to_csv(df_plans, 'Plans')

In [ ]:
pandas_df_medicalbenefits = pd.DataFrame(columns = ['QID', 'PlanYear', 'CategoryName', 'ServiceName', 'Network', 'CostShare'])

# add to pandas_df_medicalbenefits if there is no mismatching
def add_category_benefit_to_df_medicalbenefits(df, df_all):
    df_new_category_service_network = df[['CategoryName', 'ServiceName', 'Network']].drop_duplicates()
    for index, csn in df_new_category_service_network.iterrows():
        category_name = csn['CategoryName']
        service_name = csn['ServiceName']
        network = csn['Network']
        df_existing = df_all[(df_all['CategoryName'] == category_name) &  (df_all['CategoryName'] == service_name)& (df_all['Network'] == network)]
        if len(df_existing) > 0:
            df_all.drop(df_existing.index, inplace=True)
    df_selected = df[df_all.columns.tolist()]
    df_all = pd.concat([df_all, df_selected], ignore_index=True)
    return df_all


In [ ]:
# Source:  [usp_migrate_cms_23_pbpBenefit_2023]
# Implements Medical Benefits
# Health Deducutible Limit
'''
Test Cases:
https://mymedicarebot.atlassian.net/browse/MYR-6021
'''
# Category 1 Deductible  Health Deductible
'''
REF: 
https://mymedicarebot.atlassian.net/browse/MYR-6021
https://mymedicarebot.atlassian.net/browse/MYR-5592

	- Health Deducitble Limit for Plan Table (Not Using)
	- Health Deductible Benefit Text to match medicare.gov
'''

query = '''
select 
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
pbp_d_ann_deduct_yn, pbp_d_ann_deduct_amt_type, pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn, pbp_d_ann_deduct_amt, 
pbp_d_ann_deduct_comb_type, pbp_d_comb_deduct_yn, pbp_d_comb_deduct_partb_yn, pbp_d_inn_deduct_yn, pbp_d_inn_deduct_partb_yn, pbp_d_comb_deduct_amt,
pbp_d_oon_deduct_yn, pbp_d_oon_deduct_amt, pbp_d_inn_deduct_amt, pbp_d_oon_deduct_partb_yn
	from pbp_Section_D cms
	inner join pbp_mrx m on m.bid_id = cms.bid_id
	inner join pbp_Section_A a on cms.bid_id = a.bid_id
where CAST(a.pbp_a_plan_identifier1 AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(m.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

df_health_deductible_limit = spark.sql(query)
write_to_csv_file(df_health_deductible_limit, 'MedicalBenefits_HealthDeductibleLimit_DataSource')
df_medical_benefits_health_deductible_limit = read_pd_from_csv_file('MedicalBenefits_HealthDeductibleLimit_DataSource')
df_medical_benefits_health_deductible_limit['PlanYear'] = 2024
    
from PBP_Benefit_Text import  Plan # Logic implemented in Benefit Module

df_medical_benefits_health_deductible_limit['QID'] = df_medical_benefits_health_deductible_limit.apply(lambda x:Plan.get_QID(x), axis=1)
df_medical_benefits_health_deductible_limit['CategoryName'] = 'DEDUCTIBLES'
df_medical_benefits_health_deductible_limit['ServiceName'] = 'Health deductible'
df_medical_benefits_health_deductible_limit['Network'] = ''
df_medical_benefits_health_deductible_limit['CostShare'] = df_medical_benefits_health_deductible_limit.apply(lambda x: Plan.get_HealthDeductibleLimit_Text(x), axis=1)
df_medical_benefits_health_deductible_limit = drop_pbp_mrx_columns(df_medical_benefits_health_deductible_limit)
write_pd_to_csv(df_medical_benefits_health_deductible_limit, 'MedicalBenefits_HealthDeductibleLimit')

In [ ]:
# add to pandas_df_medicalbenefits if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_health_deductible_limit, pandas_df_medicalbenefits)  

In [ ]:
# Maximum Out-of-Pocket
'''
Source: HPMS..[usp_migrate_pbpBenefit] 
CategoryID = 3
'''

query = '''select 
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
pbp_d_comb_max_enr_amt_yn, pbp_d_comb_max_enr_amt, pbp_d_out_pocket_amt_yn, pbp_d_out_pocket_amt, pbp_d_oon_max_enr_oopc_yn, pbp_d_oon_max_enr_oopc_amt, pbp_d_maxenr_oopc_type,
pbp_d_maxenr_oopc_amt
FROM pbp_section_A a
INNER JOiN pbp_Section_D d ON a.bid_id = d.bid_id
inner join pbp_mrx m on m.bid_id = a.bid_id
where CAST(a.pbp_a_plan_identifier1 AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(m.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_moop = spark.sql(query)
write_to_csv_file(df_moop, 'MedicalBenefits_MOOP_DataSource')
df_medical_benefits_moop = read_pd_from_csv_file('MedicalBenefits_MOOP_DataSource')
df_medical_benefits_moop['PlanYear'] = 2024

from PBP_Benefit_Text import  Plan # Logic implemented in Benefit Module
df_medical_benefits_moop['QID'] = df_medical_benefits_moop.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_moop['CategoryName'] = 'MAXIMUM YOU PAY FOR HEALTH SERVICES'
df_medical_benefits_moop['ServiceName'] = 'Maximum you pay for health services'
df_medical_benefits_moop['Network'] = ''


df_medical_benefits_moop['CostShare'] = df_medical_benefits_moop.apply(lambda x: Plan.get_MOOP_Text(x), axis=1)
df_medical_benefits_moop = drop_pbp_mrx_columns(df_medical_benefits_moop)
write_pd_to_csv(df_medical_benefits_moop, 'MedicalBenefits_MOOP')

In [ ]:
# add to pandas_df_medicalbenefits 
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_moop, pandas_df_medicalbenefits)

In [ ]:
def explode_inn_oon_costshare(df_benefits):
    # remove OON cost share if INN cost share is "Not covered"
    df_benefits['OON_CostShare'] = df_benefits.apply(lambda x: '' if x.INN_CostShare == 'Not covered' and ('copay' in x.OON_CostShare or 'coinsurance' in x.OON_CostShare) else x.OON_CostShare, axis = 1)

    df_benefits_inn = df_benefits[(df_benefits['INN_CostShare'] != '')]
    df_benefits_inn['Network'] = df_benefits_inn.apply(lambda x: 'In-network' if x.pbp_c_pos_yn == 1 or x.pbp_c_oon_yn  == 1 else '', axis=1)
    df_benefits_inn = df_benefits_inn[['PlanYear','QID','CategoryName','ServiceName','INN_CostShare','Medicare.gov URL', 'Network']]
    df_benefits_inn.rename(columns={"INN_CostShare": "CostShare"}, inplace=True)

    df_benefits_oon = df_benefits[(df_benefits['OON_CostShare'] != '')]
    df_benefits_oon['Network'] = 'Out-of-network'
    df_benefits_oon = df_benefits_oon[['PlanYear','QID','CategoryName','ServiceName','OON_CostShare','Medicare.gov URL', 'Network']]
    df_benefits_oon.rename(columns={"OON_CostShare": "CostShare"}, inplace=True)
    return pd.concat([df_benefits_inn, df_benefits_oon])

In [ ]:

# HOSPITAL SERVICES	Inpatient hospital coverage
# Benefit Code = 1a
# In-Network & Out-of-Network

query = '''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn, pbp_b1a_bendesc_yn,pbp_b1a_cost_vary_tiers_yn,pbp_b1a_cost_vary_tier_num,pbp_b1a_cost_vary_low_tier,pbp_b1a_coins_yn,
    pbp_b1a_copay_yn,pbp_b1a_mc_coins_cstshr_yn_t1,pbp_b1a_coins_mcs_pct_t1,pbp_b1a_coins_mcs_int_num_t1,pbp_b1a_coins_mcs_pct_int1_t1,pbp_b1a_coins_mcs_bgnd_int1_t1,pbp_b1a_coins_mcs_endd_int1_t1,pbp_b1a_coins_mcs_pct_int2_t1,
    pbp_b1a_coins_mcs_bgnd_int2_t1,pbp_b1a_coins_mcs_endd_int2_t1,pbp_b1a_coins_mcs_pct_int3_t1,pbp_b1a_coins_mcs_bgnd_int3_t1,pbp_b1a_coins_mcs_endd_int3_t1,pbp_b1a_mc_copay_cstshr_yn_t1,pbp_b1a_copay_mcs_amt_t1,
    pbp_b1a_copay_mcs_int_num_t1,pbp_b1a_copay_mcs_amt_int1_t1,pbp_b1a_copay_mcs_bgnd_int1_t1,pbp_b1a_copay_mcs_endd_int1_t1,pbp_b1a_copay_mcs_amt_int2_t1,pbp_b1a_copay_mcs_bgnd_int2_t1,pbp_b1a_copay_mcs_endd_int2_t1,
    pbp_b1a_copay_mcs_amt_int3_t1,pbp_b1a_copay_mcs_bgnd_int3_t1,pbp_b1a_copay_mcs_endd_int3_t1, pbp_b1a_copay_ad_intrvl_num_t1, pbp_b1a_copay_ad_amt_int1_t1, pbp_b1a_copay_ad_bgnd_int1_t1, pbp_b1a_copay_ad_endd_int1_t1,
    pbp_b1a_mc_coins_cstshr_yn_t2,pbp_b1a_coins_mcs_pct_t2,pbp_b1a_coins_mcs_int_num_t2,pbp_b1a_coins_mcs_pct_int1_t2,pbp_b1a_coins_mcs_bgnd_int1_t2,pbp_b1a_coins_mcs_endd_int1_t2,pbp_b1a_coins_mcs_pct_int2_t2,pbp_b1a_coins_mcs_bgnd_int2_t2,
    pbp_b1a_coins_mcs_endd_int2_t2,pbp_b1a_coins_mcs_pct_int3_t2,pbp_b1a_coins_mcs_bgnd_int3_t2,pbp_b1a_coins_mcs_endd_int3_t2,pbp_b1a_mc_copay_cstshr_yn_t2,pbp_b1a_copay_mcs_amt_t2,pbp_b1a_copay_mcs_int_num_t2,
    pbp_b1a_copay_mcs_amt_int1_t2,pbp_b1a_copay_mcs_bgnd_int1_t2,pbp_b1a_copay_mcs_endd_int1_t2,pbp_b1a_copay_mcs_amt_int2_t2,pbp_b1a_copay_mcs_bgnd_int2_t2,pbp_b1a_copay_mcs_endd_int2_t2,pbp_b1a_copay_mcs_amt_int3_t2,
    pbp_b1a_copay_mcs_bgnd_int3_t2,pbp_b1a_copay_mcs_endd_int3_t2, pbp_b1a_copay_ad_intrvl_num_t2, pbp_b1a_copay_ad_amt_int1_t2, pbp_b1a_copay_ad_bgnd_int1_t2, pbp_b1a_copay_ad_endd_int1_t2, pbp_b1a_ad_cost_vary_tiers_yn,
    pbp_b1a_auth_yn, pbp_b1a_refer_yn,pbp_c_oon_coins_ihs_yn, pbp_c_oon_coins_iha_mc_cost_yn, pbp_c_oon_coins_iha_pct, pbp_c_oon_coins_iha_intrvl_num, 
    pbp_c_oon_yn,
    pbp_c_oon_coins_iha_pct_i1, pbp_c_oon_coins_iha_bgnd_i1, pbp_c_oon_coins_iha_endd_i1,
    pbp_c_oon_coins_iha_pct_i2, pbp_c_oon_coins_iha_bgnd_i2,pbp_c_oon_coins_iha_endd_i2,
    pbp_c_oon_coins_iha_pct_i3, pbp_c_oon_coins_iha_bgnd_i3,pbp_c_oon_coins_iha_endd_i3,
    pbp_c_oon_copay_ihs_yn,  pbp_c_oon_copay_iha_mc_cost_yn, pbp_c_oon_copay_iha_ps_amt,pbp_c_oon_copay_iha_intrvl_num,
    pbp_c_oon_copay_iha_amt_i1, pbp_c_oon_copay_iha_bgnd_i1, pbp_c_oon_copay_iha_endd_i1,
    pbp_c_oon_copay_iha_amt_i2, pbp_c_oon_copay_iha_bgnd_i2,pbp_c_oon_copay_iha_endd_i2,
    pbp_c_oon_copay_iha_amt_i3, pbp_c_oon_copay_iha_bgnd_i3,pbp_c_oon_copay_iha_endd_i3,
    pbp_c_pos_yn, pbp_c_pos_mc_bendesc_subcats,
    pbp_c_pos_coins_ihs_yn, pbp_c_pos_coins_iha_mc_cost_yn, pbp_c_pos_coins_iha_intrvl_num, pbp_c_pos_coins_iha_pct,
    pbp_c_pos_coins_iha_pct_i1, pbp_c_pos_coins_iha_bgnd_i1, pbp_c_pos_coins_iha_endd_i1,
    pbp_c_pos_coins_iha_pct_i2, pbp_c_pos_coins_iha_bgnd_i2,pbp_c_pos_coins_iha_endd_i2,
    pbp_c_pos_coins_iha_pct_i3, pbp_c_pos_coins_iha_bgnd_i3,pbp_c_pos_coins_iha_endd_i3,
    pbp_c_pos_copay_ihs_yn, pbp_c_pos_copay_iha_mc_cost_yn, pbp_c_pos_copay_iha_intrvl_num, pbp_c_pos_copay_iha_ps_amt,
    pbp_c_pos_copay_iha_amt_i1, pbp_c_pos_copay_iha_bgnd_i1, pbp_c_pos_copay_iha_endd_i1,
    pbp_c_pos_copay_iha_amt_i2, pbp_c_pos_copay_iha_bgnd_i2,pbp_c_pos_copay_iha_endd_i2,
    pbp_c_pos_copay_iha_amt_i3, pbp_c_pos_copay_iha_bgnd_i3,pbp_c_pos_copay_iha_endd_i3
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b1a_inpat_hosp b on c.bid_id = b.bid_id
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

# https://www.medicare.gov/coverage/inpatient-hospital-care
b1a_ORIGINAL_MEDICARE_COST = 'In 2024 the amounts for each benefit period are:<br />$1,632 deductible for days 1 through 60<br />$408 copay per day for days 61 through 90'
b1a_ORIGINAL_MEDICARE_COST_DSNP = 'In 2024 the amounts for each benefit period are $0 or:<br />$1,632 deductible for days 1 through 60<br />$408 copay per day for days 61 through 90'

from PBP_Benefit_Text import Plan, Benefit_1a # Logic implemented in Benefit Module
df_b1a_inn = spark.sql(query)
write_to_csv_file(df_b1a_inn, 'MedicalBenefits_1a_DataSource')

df_medical_benefits_1a = read_pd_from_csv_file('MedicalBenefits_1a_DataSource')
df_medical_benefits_1a['PlanYear'] = 2024
df_medical_benefits_1a['QID'] = df_medical_benefits_1a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_1a['CategoryName'] = 'HOSPITAL SERVICES'
df_medical_benefits_1a['ServiceName'] = 'Inpatient hospital coverage'
df_medical_benefits_1a['INN_CostShare'] = df_medical_benefits_1a.apply(lambda x: Benefit_1a.get_INN_text(x), axis= 1)
df_medical_benefits_1a['OON_CostShare'] = df_medical_benefits_1a.apply(lambda x: Benefit_1a.get_OON_text(x), axis= 1)
df_medical_benefits_1a['Medicare.gov URL'] = df_medical_benefits_1a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_1a = explode_inn_oon_costshare(df_medical_benefits_1a)
df_medical_benefits_1a = drop_pbp_mrx_columns(df_medical_benefits_1a)


write_pd_to_csv(df_medical_benefits_1a, 'MedicareBenefits_1a')

In [ ]:
# add to df_medical_benefits_1a if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_1a, pandas_df_medicalbenefits)   

In [ ]:
# SKILLED NURSING FACILITY	Skilled nursing facility
# Benefit Code = 2
# In-Network & Out-of-Network
query = '''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn, b.pbp_b2_bendesc_yn, b.pbp_b2_cost_vary_tiers_yn,b.pbp_b2_cost_vary_tier_num,b.pbp_b2_cost_vary_low_tier,b.pbp_b2_coins_yn,
    b.pbp_b2_copay_yn,b.pbp_b2_mc_coins_cstshr_yn_t1,b.pbp_b2_coins_mcs_pct_t1,b.pbp_b2_coins_mcs_int_num_t1,b.pbp_b2_coins_mcs_pct_int1_t1,b.pbp_b2_coins_mcs_bgnd_int1_t1,b.pbp_b2_coins_mcs_endd_int1_t1,b.pbp_b2_coins_mcs_pct_int2_t1,
    b.pbp_b2_coins_mcs_bgnd_int2_t1,b.pbp_b2_coins_mcs_endd_int2_t1,b.pbp_b2_coins_mcs_pct_int3_t1,b.pbp_b2_coins_mcs_bgnd_int3_t1,b.pbp_b2_coins_mcs_endd_int3_t1,b.pbp_b2_mc_copay_cstshr_yn_t1,b.pbp_b2_copay_mcs_amt_t1,
    b.pbp_b2_copay_mcs_int_num_t1,b.pbp_b2_copay_mcs_amt_int1_t1,b.pbp_b2_copay_mcs_bgnd_int1_t1,b.pbp_b2_copay_mcs_endd_int1_t1,b.pbp_b2_copay_mcs_amt_int2_t1,b.pbp_b2_copay_mcs_bgnd_int2_t1,b.pbp_b2_copay_mcs_endd_int2_t1,
    b.pbp_b2_copay_mcs_amt_int3_t1,b.pbp_b2_copay_mcs_bgnd_int3_t1,b.pbp_b2_copay_mcs_endd_int3_t1, b.pbp_b2_copay_ad_intrvl_num_t1, b.pbp_b2_copay_ad_amt_int1_t1, b.pbp_b2_copay_ad_bgnd_int1_t1, b.pbp_b2_copay_ad_endd_int1_t1,
    b.pbp_b2_mc_coins_cstshr_yn_t2,b.pbp_b2_coins_mcs_pct_t2,b.pbp_b2_coins_mcs_int_num_t2,b.pbp_b2_coins_mcs_pct_int1_t2,b.pbp_b2_coins_mcs_bgnd_int1_t2,b.pbp_b2_coins_mcs_endd_int1_t2,b.pbp_b2_coins_mcs_pct_int2_t2,b.pbp_b2_coins_mcs_bgnd_int2_t2,
    b.pbp_b2_coins_mcs_endd_int2_t2,b.pbp_b2_coins_mcs_pct_int3_t2,b.pbp_b2_coins_mcs_bgnd_int3_t2,b.pbp_b2_coins_mcs_endd_int3_t2,b.pbp_b2_mc_copay_cstshr_yn_t2,b.pbp_b2_copay_mcs_amt_t2,b.pbp_b2_copay_mcs_int_num_t2,
    b.pbp_b2_copay_mcs_amt_int1_t2,b.pbp_b2_copay_mcs_bgnd_int1_t2,b.pbp_b2_copay_mcs_endd_int1_t2,b.pbp_b2_copay_mcs_amt_int2_t2,b.pbp_b2_copay_mcs_bgnd_int2_t2,b.pbp_b2_copay_mcs_endd_int2_t2,b.pbp_b2_copay_mcs_amt_int3_t2,
    b.pbp_b2_copay_mcs_bgnd_int3_t2,b.pbp_b2_copay_mcs_endd_int3_t2, b.pbp_b2_copay_ad_intrvl_num_t2, b.pbp_b2_copay_ad_amt_int1_t2, b.pbp_b2_copay_ad_bgnd_int1_t2, b.pbp_b2_copay_ad_endd_int1_t2, b.pbp_b2_ad_cost_vary_tiers_yn,
    b.pbp_b2_auth_yn, b.pbp_b2_refer_yn,pbp_c_oon_coins_snf_yn, pbp_c_oon_coins_snf_mc_cost_yn, pbp_c_oon_coins_snf_pct, pbp_c_oon_coins_snf_intrvl_num, 
    pbp_c_oon_yn,
    pbp_c_oon_coins_snf_pct_i1, pbp_c_oon_coins_snf_bgnd_i1, pbp_c_oon_coins_snf_endd_i1,
    pbp_c_oon_coins_snf_pct_i2, pbp_c_oon_coins_snf_bgnd_i2,pbp_c_oon_coins_snf_endd_i2,
    pbp_c_oon_coins_snf_pct_i3, pbp_c_oon_coins_snf_bgnd_i3,pbp_c_oon_coins_snf_endd_i3,
    pbp_c_oon_copay_snf_yn,  pbp_c_oon_copay_snf_mc_cost_yn, pbp_c_oon_copay_snf_amt,pbp_c_oon_copay_snf_intrvl_num,
    pbp_c_oon_copay_snf_amt_i1, pbp_c_oon_copay_snf_bgnd_i1, pbp_c_oon_copay_snf_endd_i1,
    pbp_c_oon_copay_snf_amt_i2, pbp_c_oon_copay_snf_bgnd_i2,pbp_c_oon_copay_snf_endd_i2,
    pbp_c_oon_copay_snf_amt_i3, pbp_c_oon_copay_snf_bgnd_i3,pbp_c_oon_copay_snf_endd_i3,
    pbp_c_pos_yn, pbp_c_pos_mc_bendesc_subcats,
    pbp_c_pos_coins_snf_yn, pbp_c_pos_coins_snf_mc_cost_yn, pbp_c_pos_coins_snf_intrvl_num, pbp_c_pos_coins_snf_pct,
    pbp_c_pos_coins_snf_pct_i1, pbp_c_pos_coins_snf_bgnd_i1, pbp_c_pos_coins_snf_endd_i1,
    pbp_c_pos_coins_snf_pct_i2, pbp_c_pos_coins_snf_bgnd_i2,pbp_c_pos_coins_snf_endd_i2,
    pbp_c_pos_coins_snf_pct_i3, pbp_c_pos_coins_snf_bgnd_i3,pbp_c_pos_coins_snf_endd_i3,
    pbp_c_pos_copay_snf_yn, pbp_c_pos_copay_snf_mc_cost_yn, pbp_c_pos_copay_snf_intrvl_num, pbp_c_pos_copay_snf_amt,
    pbp_c_pos_copay_snf_amt_i1, pbp_c_pos_copay_snf_bgnd_i1, pbp_c_pos_copay_snf_endd_i1,
    pbp_c_pos_copay_snf_amt_i2, pbp_c_pos_copay_snf_bgnd_i2,pbp_c_pos_copay_snf_endd_i2,
    pbp_c_pos_copay_snf_amt_i3, pbp_c_pos_copay_snf_bgnd_i3,pbp_c_pos_copay_snf_endd_i3
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b2_snf b on c.bid_id = b.bid_id
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''


df_b2_inn = spark.sql(query)
write_to_csv_file(df_b2_inn, 'MedicalBenefits_2_DataSource')
df_medical_benefits_2 = read_pd_from_csv_file('MedicalBenefits_2_DataSource')
df_medical_benefits_2['PlanYear'] = 2024
from PBP_Benefit_Text import Plan, Benefit_2 # Logic implemented in Benefit Module
df_medical_benefits_2['QID'] = df_medical_benefits_2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_2['CategoryName'] = 'SKILLED NURSING FACILITY'
df_medical_benefits_2['ServiceName'] = 'Skilled nursing facility'

df_medical_benefits_2['INN_CostShare'] = df_medical_benefits_2.apply(lambda x: Benefit_2.get_INN_text(x), axis=1)
df_medical_benefits_2['OON_CostShare'] = df_medical_benefits_2.apply(lambda x: Benefit_2.get_OON_text(x), axis=1)
df_medical_benefits_2['Medicare.gov URL'] = df_medical_benefits_2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_2 = explode_inn_oon_costshare(df_medical_benefits_2)
df_medical_benefits_2 = drop_pbp_mrx_columns(df_medical_benefits_2)

write_pd_to_csv(df_medical_benefits_2, 'MedicalBenefits_2')

In [ ]:
# add to df_medical_benefits_1a if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_2, pandas_df_medicalbenefits)  

In [ ]:
# HOSPITAL SERVICES	Outpatient hospital coverage
# Benefit Code = 9a1
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID, 
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b9a_copay_yn,pbp_b9a_copay_ehc,pbp_b9a_copay_ohs_amt_min,pbp_b9a_copay_ohs_amt_max, 
    pbp_b9a_coins_yn,pbp_b9a_coins_ehc,pbp_b9a_coins_ohs_pct_min,pbp_b9a_coins_ohs_pct_max,
    c.pbp_c_oon_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt,
    pbp_b9a_auth_ohs_yn, pbp_b9a_refer_ohs_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b9_outpat_hosp b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;9a1;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;9a1;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b9a_inn = spark.sql(query)
write_to_csv_file(df_b9a_inn, 'MedicalBenefits_9a1_DataSource')

df_medical_benefits_9a1 = read_pd_from_csv_file('MedicalBenefits_9a1_DataSource')
df_medical_benefits_9a1['PlanYear'] = 2024
from PBP_Benefit_Text import Plan, Benefit_9a1
df_medical_benefits_9a1['QID'] = df_medical_benefits_9a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_9a1['CategoryName'] = 'HOSPITAL SERVICES'
df_medical_benefits_9a1['ServiceName'] = 'Outpatient hospital coverage'


df_medical_benefits_9a1['INN_CostShare'] = df_medical_benefits_9a1.apply(lambda x: Benefit_9a1.get_INN_text(x), axis=1)
df_medical_benefits_9a1['OON_CostShare'] = df_medical_benefits_9a1.apply(lambda x: Benefit_9a1.get_OON_text(x), axis=1)
df_medical_benefits_9a1['Medicare.gov URL'] = df_medical_benefits_9a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_9a1 = explode_inn_oon_costshare(df_medical_benefits_9a1)
df_medical_benefits_9a1 = drop_pbp_mrx_columns(df_medical_benefits_9a1)

write_pd_to_csv(df_medical_benefits_9a1,  'MedicalBenefits_9a1')

In [ ]:
# add to df_medical_benefits_9a1 if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_9a1, pandas_df_medicalbenefits)

In [ ]:
# DOCTOR SERVICES	Primary doctor visit
# Benefit Code = 7a
# In-Network & Out-of-Network
# No Prior Authorization & Referral
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7a_copay_yn,pbp_b7a_copay_amt_mc_min,pbp_b7a_copay_amt_mc_max, 
    pbp_b7a_coins_yn,pbp_b7a_coins_pct_mc_min,pbp_b7a_coins_pct_mc_max,
    c.pbp_c_oon_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7a;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;7a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b7a = spark.sql(query)
write_to_csv_file(df_b7a, 'MedicalBenefits_7a_DataSource')

from PBP_Benefit_Text import Benefit_7a

df_medical_benefits_7a = read_pd_from_csv_file('MedicalBenefits_7a_DataSource')
df_medical_benefits_7a['PlanYear'] = 2024
df_medical_benefits_7a['QID'] = df_medical_benefits_7a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7a['CategoryName'] = 'DOCTOR SERVICES'
df_medical_benefits_7a['ServiceName'] = 'Primary doctor visit'
df_medical_benefits_7a['INN_CostShare'] = df_medical_benefits_7a.apply(lambda x: Benefit_7a.get_INN_text(x), axis=1)
df_medical_benefits_7a['OON_CostShare'] = df_medical_benefits_7a.apply(lambda x: Benefit_7a.get_OON_text(x), axis=1)
df_medical_benefits_7a['Medicare.gov URL'] = df_medical_benefits_7a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_7a = explode_inn_oon_costshare(df_medical_benefits_7a)
df_medical_benefits_7a = drop_pbp_mrx_columns(df_medical_benefits_7a)

write_pd_to_csv(df_medical_benefits_7a,  'MedicalBenefits_7a')

In [ ]:
# add to df_medical_benefits_7a if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7a, pandas_df_medicalbenefits)  

In [ ]:
# DOCTOR SERVICES	Specialist visit
# Benefit Code = 7d
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7d_copay_yn,pbp_b7d_copay_amt_mc_min,pbp_b7d_copay_amt_mc_max, 
    pbp_b7d_coins_yn,pbp_b7d_coins_pct_mc_min,pbp_b7d_coins_pct_mc_max,
    pbp_b7d_auth_yn,pbp_b7d_refer_yn,
    c.pbp_c_oon_yn, coon.pbp_c_oon_out_mc_bendesc_cats,
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7d;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;7d;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b7d = spark.sql(query)
write_to_csv_file(df_b7d, 'MedicalBenefits_7d_DataSource')
df_medical_benefits_7d = read_pd_from_csv_file('MedicalBenefits_7d_DataSource')
df_medical_benefits_7d['PlanYear'] = 2024
df_medical_benefits_7d['QID'] = df_medical_benefits_7d.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7d['CategoryName'] = 'DOCTOR SERVICES'
df_medical_benefits_7d['ServiceName'] = 'Specialist visit'


from PBP_Benefit_Text import Benefit_7d
df_medical_benefits_7d['INN_CostShare'] = df_medical_benefits_7d.apply(lambda x: Benefit_7d.get_INN_text(x), axis=1)
df_medical_benefits_7d['OON_CostShare'] = df_medical_benefits_7d.apply(lambda x: Benefit_7d.get_OON_text(x), axis=1)
df_medical_benefits_7d['Medicare.gov URL'] = df_medical_benefits_7d.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7d = explode_inn_oon_costshare(df_medical_benefits_7d)
df_medical_benefits_7d = drop_pbp_mrx_columns(df_medical_benefits_7d)
write_pd_to_csv(df_medical_benefits_7d,  'MedicalBenefits_7d')

In [ ]:
# add to df_medical_benefits_7d if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7d, pandas_df_medicalbenefits)   

In [ ]:
# Acupuncture
# Benefit Code = 13a(NMC)
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b13a_bendesc_yn,
    pbp_b13a_bendesc_ehc,
    pbp_b13a_bendesc_amo,
    pbp_b13a_bendesc_lim,
    pbp_b13a_bendesc_numv,
    pbp_b13a_bendesc_per,
    pbp_b13a_bendesc_per_d,
    pbp_b13a_maxplan_yn,
    pbp_b13a_maxplan_amt,
    pbp_b13a_maxplan_per,
    pbp_b13a_maxplan_per_d,
    pbp_b13a_maxenr_yn,
    pbp_b13a_maxenr_amt,
    pbp_b13a_maxenr_per,
    pbp_b13a_maxenr_per_d,
    pbp_b13a_coins_yn,
    pbp_b13a_coins_pct_min,
    pbp_b13a_coins_pct_max,
    pbp_b13a_ded_yn,
    pbp_b13a_ded_amt,
    pbp_b13a_copay_yn,
    pbp_b13a_copay_amt_min,
    pbp_b13a_copay_amt_max,
    pbp_b13a_auth_yn,
    pbp_b13a_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn, 
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b13_other_services b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;13a;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;13a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b13a_inn = spark.sql(query)
write_to_csv_file(df_b13a_inn, 'MedicalBenefits_13a_DataSource')
df_medical_benefits_13a = read_pd_from_csv_file('MedicalBenefits_13a_DataSource')
df_medical_benefits_13a['PlanYear'] = 2024
from PBP_Benefit_Text import Plan, Benefit_13a
df_medical_benefits_13a['QID'] = df_medical_benefits_13a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_13a['CategoryName'] = 'Acupuncture'
df_medical_benefits_13a['ServiceName'] = ''

df_medical_benefits_13a['INN_CostShare'] = df_medical_benefits_13a.apply(lambda x: Benefit_13a.get_INN_text(x), axis=1)
df_medical_benefits_13a['OON_CostShare'] = df_medical_benefits_13a.apply(lambda x: Benefit_13a.get_OON_text(x), axis=1)
df_medical_benefits_13a['Medicare.gov URL'] = df_medical_benefits_13a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_13a = explode_inn_oon_costshare(df_medical_benefits_13a)
df_medical_benefits_13a = drop_pbp_mrx_columns(df_medical_benefits_13a)
write_pd_to_csv(df_medical_benefits_13a,  'MedicalBenefits_13a')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_13a, pandas_df_medicalbenefits)   

In [ ]:
# Over the counter supplemental coverage
# Benefit Code = 13b(NMC)
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b13b_bendesc_otc,
    pbp_b13b_bendesc_amo,
    pbp_b13b_maxplan_yn,
    pbp_b13b_maxplan_amt,
    pbp_b13b_otc_maxplan_per,
    pbp_b13b_maxplan_prd_yn,
    pbp_b13b_maxenr_yn,
    pbp_b13b_maxenr_amt,
    pbp_b13b_maxenr_per,
    pbp_b13b_maxenr_per_d,
    pbp_b13b_nrt_yn,
    pbp_b13b_nrt_attestation,
    pbp_b13b_coins_yn,
    pbp_b13b_coins_pct_min,
    pbp_b13b_coins_pct_max,
    pbp_b13b_ded_yn,
    pbp_b13b_ded_amt,
    pbp_b13b_copay_yn,
    pbp_b13b_copay_amt_min,
    pbp_b13b_copay_amt_max,
    pbp_b13b_cms_otc_list_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn, 
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b13_other_services b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;13b;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;13b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b13b_inn = spark.sql(query)
write_to_csv_file(df_b13b_inn, 'MedicalBenefits_13b_DataSource')
df_medical_benefits_13b = read_pd_from_csv_file('MedicalBenefits_13b_DataSource')
df_medical_benefits_13b['PlanYear'] = 2024
from PBP_Benefit_Text import Plan, Benefit_13b
df_medical_benefits_13b['QID'] = df_medical_benefits_13b.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_13b['CategoryName'] = 'Over the counter supplemental coverage'
df_medical_benefits_13b['ServiceName'] = ''

df_medical_benefits_13b['INN_CostShare'] = df_medical_benefits_13b.apply(lambda x: Benefit_13b.get_INN_text(x), axis=1)
df_medical_benefits_13b['OON_CostShare'] = df_medical_benefits_13b.apply(lambda x: Benefit_13b.get_OON_text(x), axis=1)
df_medical_benefits_13b['Medicare.gov URL'] = df_medical_benefits_13b.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_13b = explode_inn_oon_costshare(df_medical_benefits_13b)
df_medical_benefits_13b = drop_pbp_mrx_columns(df_medical_benefits_13b)
write_pd_to_csv(df_medical_benefits_13b,  'MedicalBenefits_13b')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_13b, pandas_df_medicalbenefits)   

In [ ]:
# Meals
# Benefit Code = 13c(NMC)
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b13c_bendesc_service,
    pbp_b13c_bendesc_amo,
    pbp_b13c_meal_type_chk,
    pbp_b13c_maxplan_yn,
    pbp_b13c_maxplan_amt,
    pbp_b13c_maxplan_per,
    pbp_b13c_maxplan_per_d,
    pbp_b13c_maxenr_yn,
    pbp_b13c_maxenr_amt,
    pbp_b13c_maxenr_per,
    pbp_b13c_maxenr_per_d,
    pbp_b13c_coins_yn,
    pbp_b13c_coins_pct_min,
    pbp_b13c_coins_pct_max,
    pbp_b13c_ded_yn,
    pbp_b13c_ded_amt,
    pbp_b13c_copay_yn,
    pbp_b13c_copay_amt_min,
    pbp_b13c_copay_amt_max,
    pbp_b13c_auth_yn,
    pbp_b13c_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn, 
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b13_other_services b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;13c;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;13c;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b13c_inn = spark.sql(query)
write_to_csv_file(df_b13c_inn, 'MedicalBenefits_13c_DataSource')
df_medical_benefits_13c = read_pd_from_csv_file('MedicalBenefits_13c_DataSource')
df_medical_benefits_13c['PlanYear'] = 2024
from PBP_Benefit_Text import Plan, Benefit_13c
df_medical_benefits_13c['QID'] = df_medical_benefits_13c.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_13c['CategoryName'] = 'Meals'
df_medical_benefits_13c['ServiceName'] = ''
df_medical_benefits_13c['INN_CostShare'] = df_medical_benefits_13c.apply(lambda x: Benefit_13c.get_INN_text(x), axis=1)
df_medical_benefits_13c['OON_CostShare'] = df_medical_benefits_13c.apply(lambda x: Benefit_13c.get_OON_text(x), axis=1)
df_medical_benefits_13c['Medicare.gov URL'] = df_medical_benefits_13c.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_13c = explode_inn_oon_costshare(df_medical_benefits_13c)
df_medical_benefits_13c = drop_pbp_mrx_columns(df_medical_benefits_13c)
write_pd_to_csv(df_medical_benefits_13c,  'MedicalBenefits_13c')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_13c, pandas_df_medicalbenefits)   

In [ ]:
# PREVENTIVE DENTAL	Oral exam
# Benefit Code = 16a1
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b16a_bendesc_yn,pbp_b16a_bendesc_ehc,
    pbp_b16a_coins_yn,pbp_b16a_coins_ehc,pbp_b16a_coins_cserv_sc_pov_yn, pbp_b16a_maxenr_yn, pbp_b16a_maxplan_amt,
    pbp_b16a_coins_pct_minov,pbp_b16a_coins_pct_maxov,
    pbp_b16a_coins_pct_oe,pbp_b16a_coins_pct_maxoe,
    pbp_b16a_coins_pct_pc,pbp_b16a_coins_pct_maxpc,
    pbp_b16a_coins_pct_ft,pbp_b16a_coins_pct_maxft,
    pbp_b16a_coins_pct_dx,pbp_b16a_coins_pct_maxdx,
    pbp_b16a_copay_yn,pbp_b16a_copay_ehc,pbp_b16a_copay_cserv_sc_pov_yn,
    pbp_b16a_copay_amt_ovmin,pbp_b16a_copay_amt_ovmax,
    pbp_b16a_copay_amt_oemin,pbp_b16a_copay_amt_oemax,
    pbp_b16a_copay_amt_pcmin,pbp_b16a_copay_amt_pcmax,
    pbp_b16a_copay_amt_ftmin,pbp_b16a_copay_amt_ftmax,
    pbp_b16a_copay_amt_dxmin,pbp_b16a_copay_amt_dxmax,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn, 
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt,
    pbp_b16a_bendesc_lim_oe,
    pbp_b16a_bendesc_lim_pc,
    pbp_b16a_bendesc_lim_ft,
    pbp_b16a_bendesc_lim_dx,
    pbp_b16a_maxplan_yn,
    pbp_b16a_auth_yn, pbp_b16a_refer_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b16_dental b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and (concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;16a;%' or concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;16a;%')
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and (concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;16a;%' or concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;16a;%')
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b16a_inn = spark.sql(query)
write_to_csv_file(df_b16a_inn, 'MedicalBenefits_16a_DataSource')
df_medical_benefits_16a1 = read_pd_from_csv_file('MedicalBenefits_16a_DataSource')
df_medical_benefits_16a1['PlanYear'] = 2024
df_medical_benefits_16a1['QID'] = df_medical_benefits_16a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_16a1['CategoryName'] = 'PREVENTIVE DENTAL'
df_medical_benefits_16a1['ServiceName'] = 'Oral exam'

from PBP_Benefit_Text import Benefit_16a1
df_medical_benefits_16a1['INN_CostShare'] = df_medical_benefits_16a1.apply(lambda x: Benefit_16a1.get_INN_text(x), axis=1)
df_medical_benefits_16a1['OON_CostShare'] = df_medical_benefits_16a1.apply(lambda x: Benefit_16a1.get_OON_text(x), axis=1)
df_medical_benefits_16a1['Medicare.gov URL'] = df_medical_benefits_16a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_16a1 = explode_inn_oon_costshare(df_medical_benefits_16a1)
df_medical_benefits_16a1 = drop_pbp_mrx_columns(df_medical_benefits_16a1)
write_pd_to_csv(df_medical_benefits_16a1,  'MedicalBenefits_16a1')

In [ ]:
# add to df_medical_benefits_16a1 if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_16a1, pandas_df_medicalbenefits)   

In [ ]:
# PREVENTIVE DENTAL	Cleaning
# Benefit Code = 16a2
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b16a_bendesc_yn,pbp_b16a_bendesc_ehc,
    pbp_b16a_coins_yn,pbp_b16a_coins_ehc,pbp_b16a_coins_cserv_sc_pov_yn, pbp_b16a_maxenr_yn,pbp_b16a_maxplan_amt,
    pbp_b16a_coins_pct_minov,pbp_b16a_coins_pct_maxov,
    pbp_b16a_coins_pct_oe,pbp_b16a_coins_pct_maxoe,
    pbp_b16a_coins_pct_pc,pbp_b16a_coins_pct_maxpc,
    pbp_b16a_coins_pct_ft,pbp_b16a_coins_pct_maxft,
    pbp_b16a_coins_pct_dx,pbp_b16a_coins_pct_maxdx,
    pbp_b16a_copay_yn,pbp_b16a_copay_ehc,pbp_b16a_copay_cserv_sc_pov_yn,
    pbp_b16a_copay_amt_ovmin,pbp_b16a_copay_amt_ovmax,
    pbp_b16a_copay_amt_oemin,pbp_b16a_copay_amt_oemax,
    pbp_b16a_copay_amt_pcmin,pbp_b16a_copay_amt_pcmax,
    pbp_b16a_copay_amt_ftmin,pbp_b16a_copay_amt_ftmax,
    pbp_b16a_copay_amt_dxmin,pbp_b16a_copay_amt_dxmax,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt,
    pbp_b16a_bendesc_lim_oe,
    pbp_b16a_bendesc_lim_pc,
    pbp_b16a_bendesc_lim_ft,
    pbp_b16a_bendesc_lim_dx,
    pbp_b16a_maxplan_yn,
    pbp_b16a_auth_yn, pbp_b16a_refer_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b16_dental b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and (concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;16a;%' or concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;16a;%')
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and (concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;16a;%' or concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;16a;%')
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b16a_inn = spark.sql(query)
write_to_csv_file(df_b16a_inn, 'MedicalBenefits_16a_DataSource')
df_medical_benefits_16a2 = read_pd_from_csv_file('MedicalBenefits_16a_DataSource')
df_medical_benefits_16a2['PlanYear'] = 2024
df_medical_benefits_16a2['QID'] = df_medical_benefits_16a2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_16a2['CategoryName'] = 'PREVENTIVE DENTAL'
df_medical_benefits_16a2['ServiceName'] = 'Cleaning'


from PBP_Benefit_Text import Benefit_16a2
df_medical_benefits_16a2['INN_CostShare'] = df_medical_benefits_16a2.apply(lambda x: Benefit_16a2.get_INN_text(x), axis=1)
df_medical_benefits_16a2['OON_CostShare'] = df_medical_benefits_16a2.apply(lambda x: Benefit_16a2.get_OON_text(x), axis=1)


df_medical_benefits_16a2['Medicare.gov URL'] = df_medical_benefits_16a2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_16a2 = explode_inn_oon_costshare(df_medical_benefits_16a2)
df_medical_benefits_16a2 = drop_pbp_mrx_columns(df_medical_benefits_16a2)
write_pd_to_csv(df_medical_benefits_16a2,  'MedicalBenefits_16a2')

In [ ]:
# add to df_medical_benefits_16a2 if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_16a2, pandas_df_medicalbenefits)   

In [ ]:
# PREVENTIVE DENTAL	Fluoride treatment
# Benefit Code = 16a3
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b16a_bendesc_yn,pbp_b16a_bendesc_ehc,
    pbp_b16a_coins_yn,pbp_b16a_coins_ehc,pbp_b16a_coins_cserv_sc_pov_yn, pbp_b16a_maxenr_yn,pbp_b16a_maxplan_amt,
    pbp_b16a_coins_pct_minov,pbp_b16a_coins_pct_maxov,
    pbp_b16a_coins_pct_oe,pbp_b16a_coins_pct_maxoe,
    pbp_b16a_coins_pct_pc,pbp_b16a_coins_pct_maxpc,
    pbp_b16a_coins_pct_ft,pbp_b16a_coins_pct_maxft,
    pbp_b16a_coins_pct_dx,pbp_b16a_coins_pct_maxdx,
    pbp_b16a_copay_yn,pbp_b16a_copay_ehc,pbp_b16a_copay_cserv_sc_pov_yn,
    pbp_b16a_copay_amt_ovmin,pbp_b16a_copay_amt_ovmax,
    pbp_b16a_copay_amt_oemin,pbp_b16a_copay_amt_oemax,
    pbp_b16a_copay_amt_pcmin,pbp_b16a_copay_amt_pcmax,
    pbp_b16a_copay_amt_ftmin,pbp_b16a_copay_amt_ftmax,
    pbp_b16a_copay_amt_dxmin,pbp_b16a_copay_amt_dxmax,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt,
    pbp_b16a_bendesc_lim_oe,
    pbp_b16a_bendesc_lim_pc,
    pbp_b16a_bendesc_lim_ft,
    pbp_b16a_bendesc_lim_dx,
    pbp_b16a_maxplan_yn,
    pbp_b16a_auth_yn, pbp_b16a_refer_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b16_dental b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and (concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;16a;%' or concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;16a;%')
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and (concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;16a;%' or concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;16a;%')
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b16a_inn = spark.sql(query)
write_to_csv_file(df_b16a_inn, 'MedicalBenefits_16a_DataSource')
df_medical_benefits_16a3 = read_pd_from_csv_file('MedicalBenefits_16a_DataSource')
df_medical_benefits_16a3['PlanYear'] = 2024
df_medical_benefits_16a3['QID'] = df_medical_benefits_16a3.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_16a3['CategoryName'] = 'PREVENTIVE DENTAL'
df_medical_benefits_16a3['ServiceName'] = 'Fluoride treatment'

from PBP_Benefit_Text import Benefit_16a3
df_medical_benefits_16a3['INN_CostShare'] = df_medical_benefits_16a3.apply(lambda x: Benefit_16a3.get_INN_text(x), axis=1)
df_medical_benefits_16a3['OON_CostShare'] = df_medical_benefits_16a3.apply(lambda x: Benefit_16a3.get_OON_text(x), axis=1)

df_medical_benefits_16a3['Medicare.gov URL'] = df_medical_benefits_16a3.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_16a3 = explode_inn_oon_costshare(df_medical_benefits_16a3)
df_medical_benefits_16a3 = drop_pbp_mrx_columns(df_medical_benefits_16a3)

write_pd_to_csv(df_medical_benefits_16a3,  'MedicalBenefits_16a3')

In [ ]:
# add to df_medical_benefits_16a3 if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_16a3, pandas_df_medicalbenefits)   

In [ ]:
# PREVENTIVE DENTAL	Dental X-Rays
# Benefit Code = 16a4
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b16a_bendesc_yn,pbp_b16a_bendesc_ehc,
    pbp_b16a_coins_yn,pbp_b16a_coins_ehc,pbp_b16a_coins_cserv_sc_pov_yn, pbp_b16a_maxenr_yn,pbp_b16a_maxplan_amt,
    pbp_b16a_coins_pct_minov,pbp_b16a_coins_pct_maxov,
    pbp_b16a_coins_pct_oe,pbp_b16a_coins_pct_maxoe,
    pbp_b16a_coins_pct_pc,pbp_b16a_coins_pct_maxpc,
    pbp_b16a_coins_pct_ft,pbp_b16a_coins_pct_maxft,
    pbp_b16a_coins_pct_dx,pbp_b16a_coins_pct_maxdx,
    pbp_b16a_copay_yn,pbp_b16a_copay_ehc,pbp_b16a_copay_cserv_sc_pov_yn,
    pbp_b16a_copay_amt_ovmin,pbp_b16a_copay_amt_ovmax,
    pbp_b16a_copay_amt_oemin,pbp_b16a_copay_amt_oemax,
    pbp_b16a_copay_amt_pcmin,pbp_b16a_copay_amt_pcmax,
    pbp_b16a_copay_amt_ftmin,pbp_b16a_copay_amt_ftmax,
    pbp_b16a_copay_amt_dxmin,pbp_b16a_copay_amt_dxmax,
    pbp_b16a_bendesc_lim_oe,
    pbp_b16a_bendesc_lim_pc,
    pbp_b16a_bendesc_lim_ft,
    pbp_b16a_bendesc_lim_dx,
    pbp_b16a_maxplan_yn,
    pbp_b16a_auth_yn, pbp_b16a_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b16_dental b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and (concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;16a;%' or concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;16a;%')
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and (concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;16a;%' or concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;16a;%')
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b16a_inn = spark.sql(query)
write_to_csv_file(df_b16a_inn, 'MedicalBenefits_16a_DataSource')
df_medical_benefits_16a4 = read_pd_from_csv_file('MedicalBenefits_16a_DataSource')
df_medical_benefits_16a4['PlanYear'] = 2024
df_medical_benefits_16a4['QID'] = df_medical_benefits_16a4.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_16a4['CategoryName'] = 'PREVENTIVE DENTAL'
df_medical_benefits_16a4['ServiceName'] = 'Dental x-rays'

from PBP_Benefit_Text import Benefit_16a4
df_medical_benefits_16a4['INN_CostShare'] = df_medical_benefits_16a4.apply(lambda x: Benefit_16a4.get_INN_text(x), axis=1)
df_medical_benefits_16a4['OON_CostShare'] = df_medical_benefits_16a4.apply(lambda x: Benefit_16a4.get_OON_text(x), axis=1)

df_medical_benefits_16a4['Medicare.gov URL'] = df_medical_benefits_16a4.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_16a4 = explode_inn_oon_costshare(df_medical_benefits_16a4)
df_medical_benefits_16a4 = drop_pbp_mrx_columns(df_medical_benefits_16a4)
write_pd_to_csv(df_medical_benefits_16a4,  'MedicalBenefits_16a4')

In [ ]:
# add to df_medical_benefits_16a4 if there is no mismatching
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_16a4, pandas_df_medicalbenefits)  

In [ ]:
# TESTS, LABS, & IMAGING    Diagnostic tests & procedures
# Benefit Code = 8a1
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b8a_maxenr_yn,
    pbp_b8a_maxenr_amt,
    pbp_b8a_coins_yn,pbp_b8a_coins_ehc, 
    pbp_b8a_coins_pct_dmc,pbp_b8a_coins_pct_dmc_max,
    pbp_b8a_coins_pct_lab,pbp_b8a_coins_pct_lab_max,
    pbp_b8a_copay_yn, pbp_b8a_copay_ehc,
    pbp_b8a_copay_min_dmc_amt,pbp_b8a_copay_max_dmc_amt,
    pbp_b8a_lab_copay_amt,pbp_b8a_lab_copay_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b8_clin_diag_ther b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;8a1;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;8a1;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b8a1_inn = spark.sql(query)
write_to_csv_file(df_b8a1_inn, 'MedicalBenefits_8a1_DataSource')

df_medical_benefits_8a1 = read_pd_from_csv_file('MedicalBenefits_8a1_DataSource')
df_medical_benefits_8a1['PlanYear'] = 2024
df_medical_benefits_8a1['QID'] = df_medical_benefits_8a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_8a1['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_8a1['ServiceName'] = 'Diagnostic tests & procedures'

from PBP_Benefit_Text import Benefit_8a1

df_medical_benefits_8a1['INN_CostShare'] = df_medical_benefits_8a1.apply(lambda x: Benefit_8a1.get_INN_text(x), axis=1)
df_medical_benefits_8a1['OON_CostShare'] = df_medical_benefits_8a1.apply(lambda x: Benefit_8a1.get_OON_text(x), axis=1)
df_medical_benefits_8a1['Medicare.gov URL'] = df_medical_benefits_8a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_8a1 = explode_inn_oon_costshare(df_medical_benefits_8a1)
df_medical_benefits_8a1 = drop_pbp_mrx_columns(df_medical_benefits_8a1)

write_pd_to_csv(df_medical_benefits_8a1,  'MedicalBenefits_8a1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_8a1, pandas_df_medicalbenefits)    

In [ ]:
# TESTS, LABS, & IMAGING    Lab services
# Benefit Code = 8a2
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b8a_maxenr_yn,
    pbp_b8a_maxenr_amt,
    pbp_b8a_coins_yn,pbp_b8a_coins_ehc, 
    pbp_b8a_coins_pct_dmc,pbp_b8a_coins_pct_dmc_max,
    pbp_b8a_coins_pct_lab,pbp_b8a_coins_pct_lab_max,
    pbp_b8a_copay_yn, pbp_b8a_copay_ehc,
    pbp_b8a_copay_min_dmc_amt,pbp_b8a_copay_max_dmc_amt,
    pbp_b8a_lab_copay_amt,pbp_b8a_lab_copay_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b8_clin_diag_ther b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;8a2;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;8a2;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b8a2_inn = spark.sql(query)
write_to_csv_file(df_b8a2_inn, 'MedicalBenefits_8a2_DataSource')

df_medical_benefits_8a2 = read_pd_from_csv_file('MedicalBenefits_8a2_DataSource')
df_medical_benefits_8a2['PlanYear'] = 2024
df_medical_benefits_8a2['QID'] = df_medical_benefits_8a2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_8a2['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_8a2['ServiceName'] = 'Lab services'

from PBP_Benefit_Text import Benefit_8a2

df_medical_benefits_8a2['INN_CostShare'] = df_medical_benefits_8a2.apply(lambda x: Benefit_8a2.get_INN_text(x), axis=1)
df_medical_benefits_8a2['OON_CostShare'] = df_medical_benefits_8a2.apply(lambda x: Benefit_8a2.get_OON_text(x), axis=1)
df_medical_benefits_8a2['Medicare.gov URL'] = df_medical_benefits_8a2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_8a2 = explode_inn_oon_costshare(df_medical_benefits_8a2)
df_medical_benefits_8a2 = drop_pbp_mrx_columns(df_medical_benefits_8a2)

write_pd_to_csv(df_medical_benefits_8a2,  'MedicalBenefits_8a2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_8a2, pandas_df_medicalbenefits)   

In [ ]:
# TESTS, LABS, & IMAGING    Diagnostic radiology services (like MRI)
# Benefit Code = 8b1
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b8b_maxenr_yn,
    pbp_b8b_maxenr_amt,
    pbp_b8b_coins_yn,pbp_b8b_coins_ehc, 
    pbp_b8b_coins_pct_drs,pbp_b8b_coins_pct_drs_max,
    pbp_b8b_coins_pct_cmc,pbp_b8b_coins_pct_cmc_max,
    pbp_b8b_copay_yn, pbp_b8b_copay_ehc,
    pbp_b8b_copay_amt_drs,pbp_b8b_copay_amt_drs_max,
    pbp_b8b_copay_mc_amt, pbp_b8b_copay_mc_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b8_clin_diag_ther b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;8b1;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;8b1;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b8b1_inn = spark.sql(query)
write_to_csv_file(df_b8b1_inn, 'MedicalBenefits_8b1_DataSource')

df_medical_benefits_8b1 = read_pd_from_csv_file('MedicalBenefits_8b1_DataSource')
df_medical_benefits_8b1['PlanYear'] = 2024
df_medical_benefits_8b1['QID'] = df_medical_benefits_8b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_8b1['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_8b1['ServiceName'] = 'Diagnostic radiology services (like MRI)'

from PBP_Benefit_Text import Benefit_8b1

df_medical_benefits_8b1['INN_CostShare'] = df_medical_benefits_8b1.apply(lambda x: Benefit_8b1.get_INN_text(x), axis=1)
df_medical_benefits_8b1['OON_CostShare'] = df_medical_benefits_8b1.apply(lambda x: Benefit_8b1.get_OON_text(x), axis=1)
df_medical_benefits_8b1['Medicare.gov URL'] = df_medical_benefits_8b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_8b1 = explode_inn_oon_costshare(df_medical_benefits_8b1)
df_medical_benefits_8b1 = drop_pbp_mrx_columns(df_medical_benefits_8b1)

write_pd_to_csv(df_medical_benefits_8b1,  'MedicalBenefits_8b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_8b1, pandas_df_medicalbenefits)   

In [ ]:
# TESTS, LABS, & IMAGING    Outpatient x-rays
# Benefit Code = 8b3
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b8b_maxenr_yn,
    pbp_b8b_maxenr_amt,
    pbp_b8b_coins_yn,pbp_b8b_coins_ehc, 
    pbp_b8b_coins_pct_drs,pbp_b8b_coins_pct_drs_max,
    pbp_b8b_coins_pct_cmc,pbp_b8b_coins_pct_cmc_max,
    pbp_b8b_copay_yn, pbp_b8b_copay_ehc,
    pbp_b8b_copay_amt_drs,pbp_b8b_copay_amt_drs_max,
    pbp_b8b_copay_mc_amt, pbp_b8b_copay_mc_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b8_clin_diag_ther b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;8b3;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;8b3;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b8b3_inn = spark.sql(query)
write_to_csv_file(df_b8b3_inn, 'MedicalBenefits_8b3_DataSource')

df_medical_benefits_8b3 = read_pd_from_csv_file('MedicalBenefits_8b3_DataSource')
df_medical_benefits_8b3['PlanYear'] = 2024
df_medical_benefits_8b3['QID'] = df_medical_benefits_8b3.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_8b3['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_8b3['ServiceName'] = 'Outpatient x-rays'

from PBP_Benefit_Text import Benefit_8b3

df_medical_benefits_8b3['INN_CostShare'] = df_medical_benefits_8b3.apply(lambda x: Benefit_8b3.get_INN_text(x), axis=1)
df_medical_benefits_8b3['OON_CostShare'] = df_medical_benefits_8b3.apply(lambda x: Benefit_8b3.get_OON_text(x), axis=1)
df_medical_benefits_8b3['Medicare.gov URL'] = df_medical_benefits_8b3.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_8b3 = explode_inn_oon_costshare(df_medical_benefits_8b3)
df_medical_benefits_8b3 = drop_pbp_mrx_columns(df_medical_benefits_8b3)

write_pd_to_csv(df_medical_benefits_8b3,  'MedicalBenefits_8b3')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_8b3, pandas_df_medicalbenefits)   

In [ ]:
# TESTS, LABS, & IMAGING	Emergency care
# Benefit Code = 4a
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,   
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b4a_maxenr_yn,
    pbp_b4a_maxenr_amt,
    pbp_b4a_coins_yn,pbp_b4a_coins_pct_mc_min,pbp_b4a_coins_pct_mc_max,
    pbp_b4a_copay_yn,
    pbp_b4a_copay_amt_mc_min,pbp_b4a_copay_amt_mc_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b4_emerg_urgent b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;4a;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;4a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b4a_inn = spark.sql(query)
write_to_csv_file(df_b4a_inn, 'MedicalBenefits_4a_DataSource')

df_medical_benefits_4a = read_pd_from_csv_file('MedicalBenefits_4a_DataSource')
df_medical_benefits_4a['PlanYear'] = 2024
df_medical_benefits_4a['QID'] = df_medical_benefits_4a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_4a['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_4a['ServiceName'] = 'Emergency care'

from PBP_Benefit_Text import Benefit_4a

df_medical_benefits_4a['INN_CostShare'] = df_medical_benefits_4a.apply(lambda x: Benefit_4a.get_INN_text(x), axis=1)
df_medical_benefits_4a['OON_CostShare'] = df_medical_benefits_4a.apply(lambda x: Benefit_4a.get_OON_text(x), axis=1)
df_medical_benefits_4a['Medicare.gov URL'] = df_medical_benefits_4a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_4a = explode_inn_oon_costshare(df_medical_benefits_4a)
df_medical_benefits_4a = drop_pbp_mrx_columns(df_medical_benefits_4a)
df_medical_benefits_4a['Network'] = ''

write_pd_to_csv(df_medical_benefits_4a,  'MedicalBenefits_4a')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_4a, pandas_df_medicalbenefits)   

In [ ]:
# TESTS, LABS, & IMAGING	Urgent care
# Benefit Code = 4b
# In-Network only 
# Network is blank

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b4b_maxenr_yn,
    pbp_b4b_maxenr_amt,
    pbp_b4b_coins_yn,pbp_b4b_coins_pct_mc_min,pbp_b4b_coins_pct_mc_max,
    pbp_b4b_copay_yn,
    pbp_b4b_copay_amt_mc_min,pbp_b4b_copay_amt_mc_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b4_emerg_urgent b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;4b;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;4b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b4a_inn = spark.sql(query)
write_to_csv_file(df_b4a_inn, 'MedicalBenefits_4b_DataSource')

df_medical_benefits_4b = read_pd_from_csv_file('MedicalBenefits_4b_DataSource')
df_medical_benefits_4b['PlanYear'] = 2024
df_medical_benefits_4b['QID'] = df_medical_benefits_4b.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_4b['CategoryName'] = 'TESTS, LABS, & IMAGING'
df_medical_benefits_4b['ServiceName'] = 'Urgent care'
from PBP_Benefit_Text import Benefit_4b

df_medical_benefits_4b['INN_CostShare'] = df_medical_benefits_4b.apply(lambda x: Benefit_4b.get_INN_text(x), axis=1)
df_medical_benefits_4b['OON_CostShare'] = df_medical_benefits_4b.apply(lambda x: Benefit_4b.get_OON_text(x), axis=1)
df_medical_benefits_4b['Medicare.gov URL'] = df_medical_benefits_4b.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_4b = explode_inn_oon_costshare(df_medical_benefits_4b)
df_medical_benefits_4b = drop_pbp_mrx_columns(df_medical_benefits_4b)
df_medical_benefits_4b['Network'] = ''
write_pd_to_csv(df_medical_benefits_4b,  'MedicalBenefits_4b')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_4b, pandas_df_medicalbenefits)   

In [ ]:
# PREVENTIVE SERVICES	Preventive services
# Benefit Code = 14a
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,   
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14a_mc_prevent_attest,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;14a;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;14a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b14aa_inn = spark.sql(query)
write_to_csv_file(df_b14aa_inn, 'MedicalBenefits_14a_DataSource')

df_medical_benefits_14a = read_pd_from_csv_file('MedicalBenefits_14a_DataSource')
df_medical_benefits_14a['PlanYear'] = 2024
df_medical_benefits_14a['QID'] = df_medical_benefits_14a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14a['CategoryName'] = 'PREVENTIVE SERVICES'
df_medical_benefits_14a['ServiceName'] = 'Preventive services'

from PBP_Benefit_Text import Benefit_14a

df_medical_benefits_14a['INN_CostShare'] = df_medical_benefits_14a.apply(lambda x: Benefit_14a.get_INN_text(x), axis=1)
df_medical_benefits_14a['OON_CostShare'] = df_medical_benefits_14a.apply(lambda x: Benefit_14a.get_OON_text(x), axis=1)
df_medical_benefits_14a['Medicare.gov URL'] = df_medical_benefits_14a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14a = explode_inn_oon_costshare(df_medical_benefits_14a)
df_medical_benefits_14a = drop_pbp_mrx_columns(df_medical_benefits_14a)

write_pd_to_csv(df_medical_benefits_14a,  'MedicalBenefits_14a')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14a, pandas_df_medicalbenefits)   

In [ ]:
# Fitness
# Benefit Code = 14c4(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,

    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c4;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c4;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c4, Plan
df_medical_benefits_14c4 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c4['PlanYear'] = 2024
df_medical_benefits_14c4['QID'] = df_medical_benefits_14c4.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c4['CategoryName'] = 'Fitness'
df_medical_benefits_14c4['ServiceName'] = ''

df_medical_benefits_14c4['INN_CostShare'] = df_medical_benefits_14c4.apply(lambda x: Benefit_14c4.get_INN_text(x), axis=1)
df_medical_benefits_14c4['OON_CostShare'] = df_medical_benefits_14c4.apply(lambda x: Benefit_14c4.get_OON_text(x), axis=1)
df_medical_benefits_14c4['Medicare.gov URL'] = df_medical_benefits_14c4.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c4 = explode_inn_oon_costshare(df_medical_benefits_14c4)
df_medical_benefits_14c4 = drop_pbp_mrx_columns(df_medical_benefits_14c4)
write_pd_to_csv(df_medical_benefits_14c4,  'MedicalBenefits_14c4')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c4, pandas_df_medicalbenefits)   

In [ ]:
# Nursing hotline
# Benefit Code = 14c7(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,

    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c7;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c7;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
-- and a.bid_id = 'H2320_022_1'
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c7, Plan
df_medical_benefits_14c7 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c7['PlanYear'] = 2024
df_medical_benefits_14c7['QID'] = df_medical_benefits_14c7.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c7['CategoryName'] = 'Nursing hotline'
df_medical_benefits_14c7['ServiceName'] = ''

df_medical_benefits_14c7['INN_CostShare'] = df_medical_benefits_14c7.apply(lambda x: Benefit_14c7.get_INN_text(x), axis=1)
df_medical_benefits_14c7['OON_CostShare'] = df_medical_benefits_14c7.apply(lambda x: Benefit_14c7.get_OON_text(x), axis=1)
df_medical_benefits_14c7['Medicare.gov URL'] = df_medical_benefits_14c7.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c7 = explode_inn_oon_costshare(df_medical_benefits_14c7)
df_medical_benefits_14c7 = drop_pbp_mrx_columns(df_medical_benefits_14c7)
write_pd_to_csv(df_medical_benefits_14c7,  'MedicalBenefits_14c7')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c7, pandas_df_medicalbenefits)   

In [ ]:
# In-home Assistance
# Benefit Code = 14c10(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,

    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c10;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c10;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c10, Plan
df_medical_benefits_14c10 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c10['PlanYear'] = 2024
df_medical_benefits_14c10['QID'] = df_medical_benefits_14c10.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c10['CategoryName'] = 'In-home Assistance'
df_medical_benefits_14c10['ServiceName'] = ''

df_medical_benefits_14c10['INN_CostShare'] = df_medical_benefits_14c10.apply(lambda x: Benefit_14c10.get_INN_text(x), axis=1)
df_medical_benefits_14c10['OON_CostShare'] = df_medical_benefits_14c10.apply(lambda x: Benefit_14c10.get_OON_text(x), axis=1)
df_medical_benefits_14c10['Medicare.gov URL'] = df_medical_benefits_14c10.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c10 = explode_inn_oon_costshare(df_medical_benefits_14c10)
df_medical_benefits_14c10 = drop_pbp_mrx_columns(df_medical_benefits_14c10)
write_pd_to_csv(df_medical_benefits_14c10,  'MedicalBenefits_14c10')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c10, pandas_df_medicalbenefits)   

In [ ]:
# Emergency Response Device
# Benefit Code = 14c11(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,
pbp_b14c_bendesc_amo_prs,
pbp_b14c_maxplan_amt_prs,
pbp_b14c_maxplan_per_prs,
pbp_b14c_maxplan_per_prs_d,
pbp_b14c_maxenr_amt_prs,
pbp_b14c_maxenr_per_prs,
pbp_b14c_maxenr_per_prs_d,
pbp_b14c_coins_pct_min_prs,
pbp_b14c_coins_pct_max_prs,
pbp_b14c_copay_prs_min_amt,
pbp_b14c_copay_prs_max_amt,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c11;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c11;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c11, Plan
df_medical_benefits_14c11 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c11['PlanYear'] = 2024
df_medical_benefits_14c11['QID'] = df_medical_benefits_14c11.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c11['CategoryName'] = 'Emergency Response Device'
df_medical_benefits_14c11['ServiceName'] = ''

df_medical_benefits_14c11['INN_CostShare'] = df_medical_benefits_14c11.apply(lambda x: Benefit_14c11.get_INN_text(x), axis=1)
df_medical_benefits_14c11['OON_CostShare'] = df_medical_benefits_14c11.apply(lambda x: Benefit_14c11.get_OON_text(x), axis=1)
df_medical_benefits_14c11['Medicare.gov URL'] = df_medical_benefits_14c11.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c11 = explode_inn_oon_costshare(df_medical_benefits_14c11)
df_medical_benefits_14c11 = drop_pbp_mrx_columns(df_medical_benefits_14c11)
write_pd_to_csv(df_medical_benefits_14c11,  'MedicalBenefits_14c11')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c11, pandas_df_medicalbenefits)   

In [ ]:
# In-home Support Services
# Benefit Code = 14c21(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,

    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c21;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c21;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c21, Plan

df_medical_benefits_14c21 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c21['PlanYear'] = 2024
df_medical_benefits_14c21['QID'] = df_medical_benefits_14c21.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c21['CategoryName'] = 'In-home Support Services'
df_medical_benefits_14c21['ServiceName'] = ''

df_medical_benefits_14c21['INN_CostShare'] = df_medical_benefits_14c21.apply(lambda x: Benefit_14c21.get_INN_text(x), axis=1)
df_medical_benefits_14c21['OON_CostShare'] = df_medical_benefits_14c21.apply(lambda x: Benefit_14c21.get_OON_text(x), axis=1)
df_medical_benefits_14c21['Medicare.gov URL'] = df_medical_benefits_14c21.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c21 = explode_inn_oon_costshare(df_medical_benefits_14c21)
df_medical_benefits_14c21 = drop_pbp_mrx_columns(df_medical_benefits_14c21)
write_pd_to_csv(df_medical_benefits_14c21,  'MedicalBenefits_14c21')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c21, pandas_df_medicalbenefits)   

In [ ]:
# Caregiver
# Benefit Code = 14c22(NMC)
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b14c_bendesc_yn,
pbp_b14c_bendesc_ehc,
pbp_b14c_bendesc_amo_mhc,
pbp_b14c_bendesc_typ_mhc,
pbp_b14c_bendesc_amo_rat,
pbp_b14c_rat_bendesc_ehc,
pbp_b14c_bendesc_amo_isa,
pbp_b14c_bendesc_ihss,
pbp_b14c_bendesc_sce,
pbp_b14c_sce_type_chk,
pbp_b14c_sce_note,
pbp_b14c_maxplan_yn,
pbp_b14c_maxplan_ehc,
pbp_b14c_maxplan_amt_mhc,
pbp_b14c_maxplan_per_mhc,
pbp_b14c_maxplan_per_mhc_d,
pbp_b14c_maxplan_amt_rat,
pbp_b14c_maxplan_per_rat,
pbp_b14c_maxplan_per_rat_d,
pbp_b14c_maxplan_amt_isa,
pbp_b14c_maxplan_per_isa,
pbp_b14c_maxplan_per_isa_d,
pbp_b14c_maxplan_amt_ihss,
pbp_b14c_maxplan_per_ihss,
pbp_b14c_maxplan_per_ihss_d,
pbp_b14c_maxplan_amt_sce,
pbp_b14c_maxplan_per_sce,
pbp_b14c_maxplan_per_sce_d,
pbp_b14c_maxenr_yn,
pbp_b14c_maxenr_ehc,
pbp_b14c_maxenr_amt_mhc,
pbp_b14c_maxenr_per_mhc,
pbp_b14c_maxenr_per_mhc_d,
pbp_b14c_maxenr_amt_rat,
pbp_b14c_maxenr_per_rat,
pbp_b14c_maxenr_per_rat_d,
pbp_b14c_maxenr_amt_isa,
pbp_b14c_maxenr_per_isa,
pbp_b14c_maxenr_per_isa_d,
pbp_b14c_maxenr_amt_ihss,
pbp_b14c_maxenr_per_ihss,
pbp_b14c_maxenr_per_ihss_d,
pbp_b14c_maxenr_amt_sce,
pbp_b14c_maxenr_per_sce,
pbp_b14c_maxenr_per_sce_d,
pbp_b14c_coins_yn,
pbp_b14c_coins_ehc,
pbp_b14c_coins_pct_min_mhc,
pbp_b14c_coins_pct_max_mhc,
pbp_b14c_coins_pct_min_rat_wp,
pbp_b14c_coins_pct_max_rat_wp,
pbp_b14c_coins_pct_min_rat_nh,
pbp_b14c_coins_pct_max_rat_nh,
pbp_b14c_coins_pct_min_isa,
pbp_b14c_coins_pct_max_isa,
pbp_b14c_coins_pct_min_ihss,
pbp_b14c_coins_pct_max_ihss,
pbp_b14c_coins_pct_min_sce,
pbp_b14c_coins_pct_max_sce,
pbp_b14c_ded_yn,
pbp_b14c_ded_amt,
pbp_b14c_copay_yn,
pbp_b14c_copay_ehc,
pbp_b14c_copay_mhc_min_amt,
pbp_b14c_copay_mhc_max_amt,
pbp_b14c_copay_rat_wp_min_amt,
pbp_b14c_copay_rat_wp_max_amt,
pbp_b14c_copay_rat_nh_min_amt,
pbp_b14c_copay_rat_nh_max_amt,
pbp_b14c_copay_isa_min_amt,
pbp_b14c_copay_isa_max_amt,
pbp_b14c_copay_min_amt_ihss,
pbp_b14c_copay_max_amt_ihss,
pbp_b14c_copay_min_amt_sce,
pbp_b14c_copay_max_amt_sce,
pbp_b14c_auth_yn,
pbp_b14c_refer_yn,

    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b14_preventive b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;14c22;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;14c22;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_14c = spark.sql(query)
write_to_csv_file(df_14c, 'MedicalBenefits_14c_DataSource')

from PBP_Benefit_Text import Benefit_14c22, Plan
df_medical_benefits_14c22 = read_pd_from_csv_file('MedicalBenefits_14c_DataSource')
df_medical_benefits_14c22['PlanYear'] = 2024
df_medical_benefits_14c22['QID'] = df_medical_benefits_14c22.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_14c22['CategoryName'] = 'Caregiver'
df_medical_benefits_14c22['ServiceName'] = ''

df_medical_benefits_14c22['INN_CostShare'] = df_medical_benefits_14c22.apply(lambda x: Benefit_14c22.get_INN_text(x), axis=1)
df_medical_benefits_14c22['OON_CostShare'] = df_medical_benefits_14c22.apply(lambda x: Benefit_14c22.get_OON_text(x), axis=1)
df_medical_benefits_14c22['Medicare.gov URL'] = df_medical_benefits_14c22.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_14c22 = explode_inn_oon_costshare(df_medical_benefits_14c22)
df_medical_benefits_14c22 = drop_pbp_mrx_columns(df_medical_benefits_14c22)
write_pd_to_csv(df_medical_benefits_14c22,  'MedicalBenefits_14c22')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_14c22, pandas_df_medicalbenefits)   

In [ ]:
# AMBULANCE	Ground ambulance
# Benefit Code = 10a1
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b10a_maxenr_yn,
    pbp_b10a_maxenr_gas_amt,
    pbp_b10a_maxenr_aas_amt,
    pbp_b10a_coins_yn,pbp_b10a_coins_ehc, 
    pbp_b10a_coins_gas_pct_min,pbp_b10a_coins_gas_pct_max,
    pbp_b10a_coins_aas_pct_min,pbp_b10a_coins_aas_pct_max,
    pbp_b10a_copay_yn, pbp_b10a_copay_ehc,
    pbp_b10a_copay_gas_amt_min,pbp_b10a_copay_gas_amt_max,
    pbp_b10a_copay_aas_amt_min,pbp_b10a_copay_aas_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b10_amb_trans b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;10a1;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;10a1;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_10a_inn = spark.sql(query)
write_to_csv_file(df_10a_inn, 'MedicalBenefits_10a_DataSource')

df_medical_benefits_10a1 = read_pd_from_csv_file('MedicalBenefits_10a_DataSource')
df_medical_benefits_10a1['PlanYear'] = 2024
df_medical_benefits_10a1['QID'] = df_medical_benefits_10a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_10a1['CategoryName'] = 'AMBULANCE'
df_medical_benefits_10a1['ServiceName'] = 'Ground ambulance'

from PBP_Benefit_Text import Benefit_10a1

df_medical_benefits_10a1['INN_CostShare'] = df_medical_benefits_10a1.apply(lambda x: Benefit_10a1.get_INN_text(x), axis=1)
df_medical_benefits_10a1['OON_CostShare'] = df_medical_benefits_10a1.apply(lambda x: Benefit_10a1.get_OON_text(x), axis=1)
df_medical_benefits_10a1['Medicare.gov URL'] = df_medical_benefits_10a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_10a1 = explode_inn_oon_costshare(df_medical_benefits_10a1)
df_medical_benefits_10a1 = drop_pbp_mrx_columns(df_medical_benefits_10a1)

write_pd_to_csv(df_medical_benefits_10a1,  'MedicalBenefits_10a1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_10a1, pandas_df_medicalbenefits)   

In [ ]:
# Transportation
# Benefit Code = 10b1(NMC)
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,    
    pbp_b10b_bendesc_yn,
    pbp_b10b_bendesc_trn,
    pbp_b10b_bendesc_lim_pal,
    pbp_b10b_bendesc_per_pal,
    pbp_b10b_bendesc_per_pal_d,
    pbp_b10b_bendesc_tt_pal,
pbp_b10b_bendesc_amt_pal,
pbp_b10b_bendesc_mt_pal,
pbp_b10b_coins_yn,
pbp_b10b_coins_pct_min,
pbp_b10b_coins_pct_max,
pbp_b10b_copay_yn,
pbp_b10b_copay_amt_min,
pbp_b10b_copay_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b10_amb_trans bmc on c.bid_id = bmc.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;10b1;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;10b1;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_10b = spark.sql(query)
write_to_csv_file(df_10b, 'MedicalBenefits_10b_DataSource')
df_medical_benefits_10b1 = read_pd_from_csv_file('MedicalBenefits_10b_DataSource')
df_medical_benefits_10b1['PlanYear'] = 2024
from PBP_Benefit_Text import Benefit_10b1, Plan
df_medical_benefits_10b1['QID'] = df_medical_benefits_10b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_10b1['CategoryName'] = 'Transportation'
df_medical_benefits_10b1['ServiceName'] = ''

df_medical_benefits_10b1['INN_CostShare'] = df_medical_benefits_10b1.apply(lambda x: Benefit_10b1.get_INN_text(x), axis=1)
df_medical_benefits_10b1['OON_CostShare'] = df_medical_benefits_10b1.apply(lambda x: Benefit_10b1.get_OON_text(x), axis=1)
df_medical_benefits_10b1['Medicare.gov URL'] = df_medical_benefits_10b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_10b1 = explode_inn_oon_costshare(df_medical_benefits_10b1)
df_medical_benefits_10b1 = drop_pbp_mrx_columns(df_medical_benefits_10b1)
write_pd_to_csv(df_medical_benefits_10b1,  'MedicalBenefits_10b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_10b1, pandas_df_medicalbenefits)   

In [ ]:
# AMBULANCE	Air ambulance
# Benefit Code = 10a2
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b10a_maxenr_yn,
    pbp_b10a_maxenr_gas_amt,
    pbp_b10a_maxenr_aas_amt,
    pbp_b10a_coins_yn,pbp_b10a_coins_ehc, 
    pbp_b10a_coins_gas_pct_min,pbp_b10a_coins_gas_pct_max,
    pbp_b10a_coins_aas_pct_min,pbp_b10a_coins_aas_pct_max,
    pbp_b10a_copay_yn, pbp_b10a_copay_ehc,
    pbp_b10a_copay_gas_amt_min,pbp_b10a_copay_gas_amt_max,
    pbp_b10a_copay_aas_amt_min,pbp_b10a_copay_aas_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b10_amb_trans b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;10a2;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;10a2;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_10a_inn = spark.sql(query)
write_to_csv_file(df_10a_inn, 'MedicalBenefits_10a_DataSource')

from PBP_Benefit_Text import Plan, Benefit_10a2

df_medical_benefits_10a2 = read_pd_from_csv_file('MedicalBenefits_10a_DataSource')
df_medical_benefits_10a2['PlanYear'] = 2024
df_medical_benefits_10a2['QID'] = df_medical_benefits_10a2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_10a2['CategoryName'] = 'AMBULANCE'
df_medical_benefits_10a2['ServiceName'] = 'Air ambulance'


df_medical_benefits_10a2['INN_CostShare'] = df_medical_benefits_10a2.apply(lambda x: Benefit_10a2.get_INN_text(x), axis=1)
df_medical_benefits_10a2['OON_CostShare'] = df_medical_benefits_10a2.apply(lambda x: Benefit_10a2.get_OON_text(x), axis=1)
df_medical_benefits_10a2['Medicare.gov URL'] = df_medical_benefits_10a2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_10a2 = explode_inn_oon_costshare(df_medical_benefits_10a2)
df_medical_benefits_10a2 = drop_pbp_mrx_columns(df_medical_benefits_10a2)

write_pd_to_csv(df_medical_benefits_10a2,  'MedicalBenefits_10a2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_10a2, pandas_df_medicalbenefits)

In [ ]:
# THERAPY SERVICES  Occupational therapy visit
# Benefit Code = 7c
# In-Network & Out-of-Network
query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7c_maxenr_yn, pbp_b7c_maxenr_amt,
    pbp_b7c_coins_yn,
    pbp_b7c_coins_pct_mc_min,pbp_b7c_coins_pct_mc_max,
    pbp_b7c_copay_yn, 
    pbp_b7c_copay_mc_amt_min,pbp_b7c_copay_mc_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7c;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7c;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_10a_inn = spark.sql(query)
write_to_csv_file(df_10a_inn, 'MedicalBenefits_7c_DataSource')

df_medical_benefits_7c = read_pd_from_csv_file('MedicalBenefits_7c_DataSource')
df_medical_benefits_7c['PlanYear'] = 2024
df_medical_benefits_7c['QID'] = df_medical_benefits_7c.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7c['CategoryName'] = 'THERAPY SERVICES'
df_medical_benefits_7c['ServiceName'] = 'Occupational therapy visit'

from PBP_Benefit_Text import Benefit_7c

df_medical_benefits_7c['INN_CostShare'] = df_medical_benefits_7c.apply(lambda x: Benefit_7c.get_INN_text(x), axis=1)
df_medical_benefits_7c['OON_CostShare'] = df_medical_benefits_7c.apply(lambda x: Benefit_7c.get_OON_text(x), axis=1)
df_medical_benefits_7c['Medicare.gov URL'] = df_medical_benefits_7c.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7c = explode_inn_oon_costshare(df_medical_benefits_7c)
df_medical_benefits_7c = drop_pbp_mrx_columns(df_medical_benefits_7c)

write_pd_to_csv(df_medical_benefits_7c,  'MedicalBenefits_7c')


In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7c, pandas_df_medicalbenefits)   

In [ ]:
# THERAPY SERVICES  Physical therapy & speech & language therapy visit
# Benefit Code = 7i
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7i_maxenr_yn, pbp_b7i_maxenr_amt,
    pbp_b7i_coins_yn,
    pbp_b7i_coins_pct_mc_min,pbp_b7i_coins_pct_mc_max,
    pbp_b7i_copay_yn, 
    pbp_b7i_copay_mc_amt_min,pbp_b7i_copay_mc_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7i;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7i;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_10a_inn = spark.sql(query)
write_to_csv_file(df_10a_inn, 'MedicalBenefits_7i_DataSource')

df_medical_benefits_7i = read_pd_from_csv_file('MedicalBenefits_7i_DataSource')
df_medical_benefits_7i['PlanYear'] = 2024
df_medical_benefits_7i['QID'] = df_medical_benefits_7i.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7i['CategoryName'] = 'THERAPY SERVICES'
df_medical_benefits_7i['ServiceName'] = 'Physical therapy & speech & language therapy visit'

from PBP_Benefit_Text import Benefit_7i

df_medical_benefits_7i['INN_CostShare'] = df_medical_benefits_7i.apply(lambda x: Benefit_7i.get_INN_text(x), axis=1)
df_medical_benefits_7i['OON_CostShare'] = df_medical_benefits_7i.apply(lambda x: Benefit_7i.get_OON_text(x), axis=1)
df_medical_benefits_7i['Medicare.gov URL'] = df_medical_benefits_7i.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7i = explode_inn_oon_costshare(df_medical_benefits_7i)
df_medical_benefits_7i = drop_pbp_mrx_columns(df_medical_benefits_7i)

write_pd_to_csv(df_medical_benefits_7i,  'MedicalBenefits_7i')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7i, pandas_df_medicalbenefits)   

In [ ]:
# THERAPY SERVICES  Physical therapy & speech & language therapy visit
# Benefit Code = 7i
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7j_bendesc_yn,
pbp_b7j_mc_bendesc_cats,
pbp_b7j_maxenr_yn,
pbp_b7j_maxenr_amt,
pbp_b7j_maxenr_per,
pbp_b7j_maxenr_per_d,
pbp_b7j_coins_yn,
pbp_b7j_coins_pct_mc_min,
pbp_b7j_coins_pct_mc_max,
pbp_b7j_ded_yn,
pbp_b7j_ded_amt,
pbp_b7j_copay_yn,
pbp_b7j_copay_mc_amt_min,
pbp_b7j_copay_mc_amt_max,
pbp_b7j_auth_yn,
pbp_b7j_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7j;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7j;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7j_inn = spark.sql(query)
write_to_csv_file(df_7j_inn, 'MedicalBenefits_7j_DataSource')

df_medical_benefits_7j = read_pd_from_csv_file('MedicalBenefits_7j_DataSource')
df_medical_benefits_7j['PlanYear'] = 2024
df_medical_benefits_7j['QID'] = df_medical_benefits_7j.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7j['CategoryName'] = 'Telehealth'
df_medical_benefits_7j['ServiceName'] = ''

from PBP_Benefit_Text import Benefit_7j

df_medical_benefits_7j['INN_CostShare'] = df_medical_benefits_7j.apply(lambda x: Benefit_7j.get_INN_text(x), axis=1)
df_medical_benefits_7j['OON_CostShare'] = df_medical_benefits_7j.apply(lambda x: Benefit_7j.get_OON_text(x), axis=1)
df_medical_benefits_7j['Medicare.gov URL'] = df_medical_benefits_7j.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7j = explode_inn_oon_costshare(df_medical_benefits_7j)
df_medical_benefits_7j = drop_pbp_mrx_columns(df_medical_benefits_7j)

write_pd_to_csv(df_medical_benefits_7j,  'MedicalBenefits_7j')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7j, pandas_df_medicalbenefits)   

In [ ]:
# Chiropractic Care
# Benefit Code = 7b
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7b_coins_yn,
    pbp_b7b_coins_ehc,
    pbp_b7b_coins_pct_mc_min,
    pbp_b7b_coins_pct_mc_max,
    pbp_b7b_copay_yn,
    pbp_b7b_copay_ehc,
    pbp_b7b_copay_mc_amt_min,
    pbp_b7b_copay_mc_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7b;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7b_inn = spark.sql(query)
write_to_csv_file(df_7b_inn, 'MedicalBenefits_7b_DataSource')

from PBP_Benefit_Text import Benefit_7b, Plan

df_medical_benefits_7b = read_pd_from_csv_file('MedicalBenefits_7b_DataSource')
df_medical_benefits_7b['PlanYear'] = 2024
df_medical_benefits_7b['QID'] = df_medical_benefits_7b.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7b['CategoryName'] = 'Chiropractic Care'
df_medical_benefits_7b['ServiceName'] = ''


df_medical_benefits_7b['INN_CostShare'] = df_medical_benefits_7b.apply(lambda x: Benefit_7b.get_INN_text(x), axis=1)
df_medical_benefits_7b['OON_CostShare'] = df_medical_benefits_7b.apply(lambda x: Benefit_7b.get_OON_text(x), axis=1)
df_medical_benefits_7b['Medicare.gov URL'] = df_medical_benefits_7b.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7b = explode_inn_oon_costshare(df_medical_benefits_7b)
df_medical_benefits_7b = drop_pbp_mrx_columns(df_medical_benefits_7b)

write_pd_to_csv(df_medical_benefits_7b,  'MedicalBenefits_7b')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7b, pandas_df_medicalbenefits)   

In [ ]:
# Chiropractic Care Routine Chiropractic Care
# Benefit Code = 7b1(NMC)
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7b_bendesc_yn,
pbp_b7b_bendesc_ehc,
pbp_b7b_bendesc_amo_rc,
pbp_b7b_bendesc_lim_rc,
pbp_b7b_bendesc_num_rc,
pbp_b7b_bendesc_per_rc,
pbp_b7b_bendesc_per_rc_d,
pbp_b7b_addl_service_other,
pbp_b7b_bendesc_amo_other,
pbp_b7b_bendesc_lim_other,
pbp_b7b_bendesc_num_other,
pbp_b7b_bendesc_per_other,
pbp_b7b_bendesc_per_other_d,
pbp_b7b_maxplan_yn,
pbp_b7b_maxplan_amt,
pbp_b7b_maxplan_per,
pbp_b7b_maxplan_per_d,
pbp_b7b_maxenr_yn,
pbp_b7b_maxenr_amt,
pbp_b7b_maxenr_per,
pbp_b7b_maxenr_per_d,
pbp_b7b_coins_yn,
pbp_b7b_coins_ehc,
pbp_b7b_coins_pct_mc_min,
pbp_b7b_coins_pct_mc_max,
pbp_b7b_coins_pct_rc_min,
pbp_b7b_coins_pct_rc_max,
pbp_b7b_coins_pct_other_min,
pbp_b7b_coins_pct_other_max,
pbp_b7b_copay_yn,
pbp_b7b_copay_ehc,
pbp_b7b_copay_mc_amt_min,
pbp_b7b_copay_mc_amt_max,
pbp_b7b_copay_rc_amt_min,
pbp_b7b_copay_rc_amt_max,
pbp_b7b_copay_other_amt_min,
pbp_b7b_copay_other_amt_max,
pbp_b7b_ded_yn,
pbp_b7b_ded_amt,
pbp_b7b_auth_yn,
pbp_b7b_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;7b;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;7b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7b_inn = spark.sql(query)
write_to_csv_file(df_7b_inn, 'MedicalBenefits_7b_DataSource')

from PBP_Benefit_Text import Benefit_7b1, Plan

df_medical_benefits_7b1 = read_pd_from_csv_file('MedicalBenefits_7b_DataSource')
df_medical_benefits_7b1['PlanYear'] = 2024
df_medical_benefits_7b1['QID'] = df_medical_benefits_7b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7b1['CategoryName'] = 'Chiropractic Care'
df_medical_benefits_7b1['ServiceName'] = 'Routine Chiropractic Care'


df_medical_benefits_7b1['INN_CostShare'] = df_medical_benefits_7b1.apply(lambda x: Benefit_7b1.get_INN_text(x), axis=1)
df_medical_benefits_7b1['OON_CostShare'] = df_medical_benefits_7b1.apply(lambda x: Benefit_7b1.get_OON_text(x), axis=1)
df_medical_benefits_7b1['Medicare.gov URL'] = df_medical_benefits_7b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7b1 = explode_inn_oon_costshare(df_medical_benefits_7b1)
df_medical_benefits_7b1 = drop_pbp_mrx_columns(df_medical_benefits_7b1)

write_pd_to_csv(df_medical_benefits_7b1,  'MedicalBenefits_7b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7b1, pandas_df_medicalbenefits)   

In [ ]:
# Chiropractic Care Other Chiropractic Care
# Benefit Code = 7b2(NMC)
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7b_bendesc_yn,
pbp_b7b_bendesc_ehc,
pbp_b7b_bendesc_amo_rc,
pbp_b7b_bendesc_lim_rc,
pbp_b7b_bendesc_num_rc,
pbp_b7b_bendesc_per_rc,
pbp_b7b_bendesc_per_rc_d,
pbp_b7b_addl_service_other,
pbp_b7b_bendesc_amo_other,
pbp_b7b_bendesc_lim_other,
pbp_b7b_bendesc_num_other,
pbp_b7b_bendesc_per_other,
pbp_b7b_bendesc_per_other_d,
pbp_b7b_maxplan_yn,
pbp_b7b_maxplan_amt,
pbp_b7b_maxplan_per,
pbp_b7b_maxplan_per_d,
pbp_b7b_maxenr_yn,
pbp_b7b_maxenr_amt,
pbp_b7b_maxenr_per,
pbp_b7b_maxenr_per_d,
pbp_b7b_coins_yn,
pbp_b7b_coins_ehc,
pbp_b7b_coins_pct_mc_min,
pbp_b7b_coins_pct_mc_max,
pbp_b7b_coins_pct_rc_min,
pbp_b7b_coins_pct_rc_max,
pbp_b7b_coins_pct_other_min,
pbp_b7b_coins_pct_other_max,
pbp_b7b_copay_yn,
pbp_b7b_copay_ehc,
pbp_b7b_copay_mc_amt_min,
pbp_b7b_copay_mc_amt_max,
pbp_b7b_copay_rc_amt_min,
pbp_b7b_copay_rc_amt_max,
pbp_b7b_copay_other_amt_min,
pbp_b7b_copay_other_amt_max,
pbp_b7b_ded_yn,
pbp_b7b_ded_amt,
pbp_b7b_auth_yn,
pbp_b7b_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;7b;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;7b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7b_inn = spark.sql(query)
write_to_csv_file(df_7b_inn, 'MedicalBenefits_7b_DataSource')

from PBP_Benefit_Text import Benefit_7b1, Plan

df_medical_benefits_7b1 = read_pd_from_csv_file('MedicalBenefits_7b_DataSource')
df_medical_benefits_7b1['PlanYear'] = 2024
df_medical_benefits_7b1['QID'] = df_medical_benefits_7b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7b1['CategoryName'] = 'Chiropractic Care'
df_medical_benefits_7b1['ServiceName'] = 'Routine Chiropractic Care'


df_medical_benefits_7b1['INN_CostShare'] = df_medical_benefits_7b1.apply(lambda x: Benefit_7b1.get_INN_text(x), axis=1)
df_medical_benefits_7b1['OON_CostShare'] = df_medical_benefits_7b1.apply(lambda x: Benefit_7b1.get_OON_text(x), axis=1)
df_medical_benefits_7b1['Medicare.gov URL'] = df_medical_benefits_7b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7b1 = explode_inn_oon_costshare(df_medical_benefits_7b1)
df_medical_benefits_7b1 = drop_pbp_mrx_columns(df_medical_benefits_7b1)

write_pd_to_csv(df_medical_benefits_7b1,  'MedicalBenefits_7b1')

In [ ]:
# Chiropractic Care Other Chiropractic Care
# Benefit Code = 7b2(NMC)
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7b_bendesc_yn,
pbp_b7b_bendesc_ehc,
pbp_b7b_bendesc_amo_rc,
pbp_b7b_bendesc_lim_rc,
pbp_b7b_bendesc_num_rc,
pbp_b7b_bendesc_per_rc,
pbp_b7b_bendesc_per_rc_d,
pbp_b7b_addl_service_other,
pbp_b7b_bendesc_amo_other,
pbp_b7b_bendesc_lim_other,
pbp_b7b_bendesc_num_other,
pbp_b7b_bendesc_per_other,
pbp_b7b_bendesc_per_other_d,
pbp_b7b_maxplan_yn,
pbp_b7b_maxplan_amt,
pbp_b7b_maxplan_per,
pbp_b7b_maxplan_per_d,
pbp_b7b_maxenr_yn,
pbp_b7b_maxenr_amt,
pbp_b7b_maxenr_per,
pbp_b7b_maxenr_per_d,
pbp_b7b_coins_yn,
pbp_b7b_coins_ehc,
pbp_b7b_coins_pct_mc_min,
pbp_b7b_coins_pct_mc_max,
pbp_b7b_coins_pct_rc_min,
pbp_b7b_coins_pct_rc_max,
pbp_b7b_coins_pct_other_min,
pbp_b7b_coins_pct_other_max,
pbp_b7b_copay_yn,
pbp_b7b_copay_ehc,
pbp_b7b_copay_mc_amt_min,
pbp_b7b_copay_mc_amt_max,
pbp_b7b_copay_rc_amt_min,
pbp_b7b_copay_rc_amt_max,
pbp_b7b_copay_other_amt_min,
pbp_b7b_copay_other_amt_max,
pbp_b7b_ded_yn,
pbp_b7b_ded_amt,
pbp_b7b_auth_yn,
pbp_b7b_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;7b;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;7b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7b_inn = spark.sql(query)
write_to_csv_file(df_7b_inn, 'MedicalBenefits_7b_DataSource')

from PBP_Benefit_Text import Benefit_7b2, Plan

df_medical_benefits_7b2 = read_pd_from_csv_file('MedicalBenefits_7b_DataSource')
df_medical_benefits_7b2['PlanYear'] = 2024
df_medical_benefits_7b2['QID'] = df_medical_benefits_7b2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7b2['CategoryName'] = 'Chiropractic Care'
df_medical_benefits_7b2['ServiceName'] = 'Other Chiropractic Care'


df_medical_benefits_7b2['INN_CostShare'] = df_medical_benefits_7b2.apply(lambda x: Benefit_7b2.get_INN_text(x), axis=1)
df_medical_benefits_7b2['OON_CostShare'] = df_medical_benefits_7b2.apply(lambda x: Benefit_7b2.get_OON_text(x), axis=1)
df_medical_benefits_7b2['Medicare.gov URL'] = df_medical_benefits_7b2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7b2 = explode_inn_oon_costshare(df_medical_benefits_7b2)
df_medical_benefits_7b2 = drop_pbp_mrx_columns(df_medical_benefits_7b2)

write_pd_to_csv(df_medical_benefits_7b2,  'MedicalBenefits_7b2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7b2, pandas_df_medicalbenefits)   

In [ ]:
# MENTAL HEALTH SERVICES	Outpatient group therapy visit
# Benefit Code = 7e2
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7e_maxenr_yn,
    pbp_b7e_maxenr_amt,
    pbp_b7e_coins_yn,
    pbp_b7e_coins_ehc,
    pbp_b7e_coins_mcis_minpct,
    pbp_b7e_coins_mcis_maxpct,
    pbp_b7e_coins_mcgs_minpct,
    pbp_b7e_coins_mcgs_maxpct,
    pbp_b7e_copay_yn,
    pbp_b7e_copay_ehc,
    pbp_b7e_copay_mcis_minamt,
    pbp_b7e_copay_mcis_maxamt,
    pbp_b7e_copay_mcgs_minamt,
    pbp_b7e_copay_mcgs_maxamt,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7e;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7e;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7e2 = spark.sql(query)
write_to_csv_file(df_7e2, 'MedicalBenefits_7e2_DataSource')

df_medical_benefits_7e2 = read_pd_from_csv_file('MedicalBenefits_7e2_DataSource')
df_medical_benefits_7e2['PlanYear'] = 2024
df_medical_benefits_7e2['QID'] = df_medical_benefits_7e2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7e2['CategoryName'] = 'MENTAL HEALTH SERVICES'
df_medical_benefits_7e2['ServiceName'] = 'Outpatient group therapy visit'

from PBP_Benefit_Text import Benefit_7e2

df_medical_benefits_7e2['INN_CostShare'] = df_medical_benefits_7e2.apply(lambda x: Benefit_7e2.get_INN_text(x), axis=1)
df_medical_benefits_7e2['OON_CostShare'] = df_medical_benefits_7e2.apply(lambda x: Benefit_7e2.get_OON_text(x), axis=1)
df_medical_benefits_7e2['Medicare.gov URL'] = df_medical_benefits_7e2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7e2 = explode_inn_oon_costshare(df_medical_benefits_7e2)
df_medical_benefits_7e2 = drop_pbp_mrx_columns(df_medical_benefits_7e2)

write_pd_to_csv(df_medical_benefits_7e2,  'MedicalBenefits_7e2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7e2, pandas_df_medicalbenefits)   

In [ ]:
# MENTAL HEALTH SERVICES    Outpatient individual therapy visit
# Benefit Code = 7e1
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7e_maxenr_yn,
    pbp_b7e_maxenr_amt,
    pbp_b7e_coins_yn,
    pbp_b7e_coins_ehc,
    pbp_b7e_coins_mcis_minpct,
    pbp_b7e_coins_mcis_maxpct,
    pbp_b7e_coins_mcgs_minpct,
    pbp_b7e_coins_mcgs_maxpct,
    pbp_b7e_copay_yn,
    pbp_b7e_copay_ehc,
    pbp_b7e_copay_mcis_minamt,
    pbp_b7e_copay_mcis_maxamt,
    pbp_b7e_copay_mcgs_minamt,
    pbp_b7e_copay_mcgs_maxamt,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7e;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7e;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7e1 = spark.sql(query)
write_to_csv_file(df_7e1, 'MedicalBenefits_7e1_DataSource')

df_medical_benefits_7e1 = read_pd_from_csv_file('MedicalBenefits_7e1_DataSource')
df_medical_benefits_7e1['PlanYear'] = 2024
df_medical_benefits_7e1['QID'] = df_medical_benefits_7e1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7e1['CategoryName'] = 'MENTAL HEALTH SERVICES'
df_medical_benefits_7e1['ServiceName'] = 'Outpatient individual therapy visit'

from PBP_Benefit_Text import Benefit_7e1
df_medical_benefits_7e1['INN_CostShare'] = df_medical_benefits_7e1.apply(lambda x: Benefit_7e1.get_INN_text(x), axis=1)
df_medical_benefits_7e1['OON_CostShare'] = df_medical_benefits_7e1.apply(lambda x: Benefit_7e1.get_OON_text(x), axis=1)
df_medical_benefits_7e1['Medicare.gov URL'] = df_medical_benefits_7e1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7e1 = explode_inn_oon_costshare(df_medical_benefits_7e1)
df_medical_benefits_7e1 = drop_pbp_mrx_columns(df_medical_benefits_7e1)

write_pd_to_csv(df_medical_benefits_7e1,  'MedicalBenefits_7e1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7e1, pandas_df_medicalbenefits)   

In [ ]:
# Foot care <i>(podiatry services)</i>	Foot exams and treatment
# Benefit Code = 7f
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7f_coins_yn,
    pbp_b7f_coins_ehc,
    pbp_b7f_coins_pct_mc_min,
    pbp_b7f_coins_pct_mc_max,
    pbp_b7f_coins_pct_rf_min,
    pbp_b7f_coins_pct_rf_max,
    pbp_b7f_copay_yn,
    pbp_b7f_copay_ehc,
    pbp_b7f_copay_mc_amt_min,
    pbp_b7f_copay_mc_amt_max,
    pbp_b7f_copay_rf_amt_min,
    pbp_b7f_copay_rf_amt_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,
    pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7f;%' 
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;7f;%' 
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b7f = spark.sql(query)
write_to_csv_file(df_b7f, 'MedicalBenefits_7f_DataSource')
df_medical_benefits_7f = read_pd_from_csv_file('MedicalBenefits_7f_DataSource')
df_medical_benefits_7f['PlanYear'] = 2024
df_medical_benefits_7f['QID'] = df_medical_benefits_7f.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7f['CategoryName'] = 'Foot care <i>(podiatry services)</i>'
df_medical_benefits_7f['ServiceName'] = 'Foot exams and treatment'


from PBP_Benefit_Text import Benefit_7f
df_medical_benefits_7f['INN_CostShare'] = df_medical_benefits_7f.apply(lambda x: Benefit_7f.get_INN_text(x), axis=1)
df_medical_benefits_7f['OON_CostShare'] = df_medical_benefits_7f.apply(lambda x: Benefit_7f.get_OON_text(x), axis=1)


df_medical_benefits_7f['Medicare.gov URL'] = df_medical_benefits_7f.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7f = explode_inn_oon_costshare(df_medical_benefits_7f)
df_medical_benefits_7f = drop_pbp_mrx_columns(df_medical_benefits_7f)
write_pd_to_csv(df_medical_benefits_7f,  'MedicalBenefits_7f')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7f, pandas_df_medicalbenefits)   

In [ ]:
# Foot care <i>(podiatry services)</i>	Routine foot care
# Benefit Code = 7f(NMC)
# In-Network & Out-of-Network

query = f'''select
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7f_coins_yn,
    pbp_b7f_coins_ehc,
    pbp_b7f_coins_pct_mc_min,
    pbp_b7f_coins_pct_mc_max,
    pbp_b7f_coins_pct_rf_min,
    pbp_b7f_coins_pct_rf_max,
    pbp_b7f_copay_yn,
    pbp_b7f_copay_ehc,
    pbp_b7f_copay_mc_amt_min,
    pbp_b7f_copay_mc_amt_max,
    pbp_b7f_copay_rf_amt_min,
    pbp_b7f_copay_rf_amt_max,
    pbp_b7f_bendesc_lim_rf,
    pbp_b7f_bendesc_amt_rf,
    pbp_b7f_bendesc_per_rf,
    c.pbp_c_oon_yn, pbp_c_pos_yn, pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,
    pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;7f_2;%' 
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_nmc_bencats, '')) like '%;7f_2;%' 
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_b7f_nmc = spark.sql(query)
write_to_csv_file(df_b7f_nmc, 'MedicalBenefits_7f_NMC_DataSource')
df_medical_benefits_7f_nmc = read_pd_from_csv_file('MedicalBenefits_7f_NMC_DataSource')
df_medical_benefits_7f_nmc['PlanYear'] = 2024
from PBP_Benefit_Text import Benefit_7f_NMC, Plan
df_medical_benefits_7f_nmc['QID'] = df_medical_benefits_7f_nmc.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7f_nmc['CategoryName'] = 'Foot care <i>(podiatry services)</i>'
df_medical_benefits_7f_nmc['ServiceName'] = 'Routine foot care'


df_medical_benefits_7f_nmc['INN_CostShare'] = df_medical_benefits_7f_nmc.apply(lambda x: Benefit_7f_NMC.get_INN_text(x), axis=1)
df_medical_benefits_7f_nmc['OON_CostShare'] = df_medical_benefits_7f_nmc.apply(lambda x: Benefit_7f_NMC.get_OON_text(x), axis=1)


df_medical_benefits_7f_nmc['Medicare.gov URL'] = df_medical_benefits_7f_nmc.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7f_nmc = explode_inn_oon_costshare(df_medical_benefits_7f_nmc)
df_medical_benefits_7f_nmc = drop_pbp_mrx_columns(df_medical_benefits_7f_nmc)
write_pd_to_csv(df_medical_benefits_7f_nmc,  'MedicalBenefits_7f_NMC')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7f_nmc, pandas_df_medicalbenefits)   

In [ ]:
# MENTAL HEALTH SERVICES	Outpatient group therapy with a psychiatrist
# Benefit Code = 7h2
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7h_maxenr_yn,
    pbp_b7h_maxenr_amt,
    pbp_b7h_coins_yn,
    pbp_b7h_coins_ehc,
    pbp_b7h_coins_mcis_minpct,
    pbp_b7h_coins_mcis_maxpct,
    pbp_b7h_coins_mcgs_minpct,
    pbp_b7h_coins_mcgs_maxpct,
    pbp_b7h_copay_yn,
    pbp_b7h_copay_ehc,
    pbp_b7h_copay_mcis_minamt,
    pbp_b7h_copay_mcis_maxamt,
    pbp_b7h_copay_mcgs_minamt,
    pbp_b7h_copay_mcgs_maxamt,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7h;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7h;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7h2 = spark.sql(query)
write_to_csv_file(df_7h2, 'MedicalBenefits_7h2_DataSource')

df_medical_benefits_7h2 = read_pd_from_csv_file('MedicalBenefits_7h2_DataSource')
df_medical_benefits_7h2['PlanYear'] = 2024
df_medical_benefits_7h2['QID'] = df_medical_benefits_7h2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7h2['CategoryName'] = 'MENTAL HEALTH SERVICES'
df_medical_benefits_7h2['ServiceName'] = 'Outpatient group therapy with a psychiatrist'

from PBP_Benefit_Text import Benefit_7h2
df_medical_benefits_7h2['INN_CostShare'] = df_medical_benefits_7h2.apply(lambda x: Benefit_7h2.get_INN_text(x), axis=1)
df_medical_benefits_7h2['OON_CostShare'] = df_medical_benefits_7h2.apply(lambda x: Benefit_7h2.get_OON_text(x), axis=1)
df_medical_benefits_7h2['Medicare.gov URL'] = df_medical_benefits_7h2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7h2 = explode_inn_oon_costshare(df_medical_benefits_7h2)
df_medical_benefits_7h2 = drop_pbp_mrx_columns(df_medical_benefits_7h2)
write_pd_to_csv(df_medical_benefits_7h2,  'MedicalBenefits_7h2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7h2, pandas_df_medicalbenefits)   

In [ ]:
# MENTAL HEALTH SERVICES	Outpatient individual therapy with a psychiatrist
# Benefit Code = 7h1
# In-Network & Out-of-Network
query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b7h_maxenr_yn,
    pbp_b7h_maxenr_amt,
    pbp_b7h_coins_yn,
    pbp_b7h_coins_ehc,
    pbp_b7h_coins_mcis_minpct,
    pbp_b7h_coins_mcis_maxpct,
    pbp_b7h_coins_mcgs_minpct,
    pbp_b7h_coins_mcgs_maxpct,
    pbp_b7h_copay_yn,
    pbp_b7h_copay_ehc,
    pbp_b7h_copay_mcis_minamt,
    pbp_b7h_copay_mcis_maxamt,
    pbp_b7h_copay_mcgs_minamt,
    pbp_b7h_copay_mcgs_maxamt,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b7_health_prof b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;7h;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;7h;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_7h1 = spark.sql(query)
write_to_csv_file(df_7h1, 'MedicalBenefits_7h1_DataSource')

df_medical_benefits_7h1 = read_pd_from_csv_file('MedicalBenefits_7h1_DataSource')
df_medical_benefits_7h1['PlanYear'] = 2024
df_medical_benefits_7h1['QID'] = df_medical_benefits_7h1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_7h1['CategoryName'] = 'MENTAL HEALTH SERVICES'
df_medical_benefits_7h1['ServiceName'] = 'Outpatient individual therapy with a psychiatrist'

from PBP_Benefit_Text import Benefit_7h1
df_medical_benefits_7h1['INN_CostShare'] = df_medical_benefits_7h1.apply(lambda x: Benefit_7h1.get_INN_text(x), axis=1)
df_medical_benefits_7h1['OON_CostShare'] = df_medical_benefits_7h1.apply(lambda x: Benefit_7h1.get_OON_text(x), axis=1)
df_medical_benefits_7h1['Medicare.gov URL'] = df_medical_benefits_7h1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_7h1 = explode_inn_oon_costshare(df_medical_benefits_7h1)
df_medical_benefits_7h1 = drop_pbp_mrx_columns(df_medical_benefits_7h1)
write_pd_to_csv(df_medical_benefits_7h1,  'MedicalBenefits_7h1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_7h1, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing exam (Medicare covered)
# Benefit Code = 18a
# In-Network & Out-of-Network
# uses Medicare for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b18a_bendesc_yn,
    pbp_b18a_bendesc_ehc,
    pbp_b18a_maxenr_yn,
    pbp_b18a_maxenr_amt,
    pbp_b18a_coins_yn,    
    pbp_b18a_coins_ehc,
    pbp_b18a_med_coins_pct,
    pbp_b18a_med_coins_pct_max,
    pbp_b18a_coins_pct_rht,
    pbp_b18a_coins_pct_max_rht,
    pbp_b18a_coins_pct_fha,
    pbp_b18a_coins_pct_max_fha,
    pbp_b18a_copay_yn,
    pbp_b18a_copay_ehc,
    pbp_b18a_copay_amt,
    pbp_b18a_med_copay_amt_max,
    pbp_b18a_copay_amt_rht,
    pbp_b18a_copay_amt_max_rht,
    pbp_b18a_copay_amt_fha,
    pbp_b18a_copay_amt_max_fha,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;18a;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;18a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18a = spark.sql(query)
write_to_csv_file(df_18a, 'MedicalBenefits_18a_DataSource')

from PBP_Benefit_Text import Benefit_18a, Plan
df_medical_benefits_18a = read_pd_from_csv_file('MedicalBenefits_18a_DataSource')
df_medical_benefits_18a['PlanYear'] = 2024
df_medical_benefits_18a['QID'] = df_medical_benefits_18a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18a['CategoryName'] = 'HEARING'
df_medical_benefits_18a['ServiceName'] = 'Hearing exam (Medicare covered)'

df_medical_benefits_18a['INN_CostShare'] = df_medical_benefits_18a.apply(lambda x: Benefit_18a.get_INN_text(x), axis=1)
df_medical_benefits_18a['OON_CostShare'] = df_medical_benefits_18a.apply(lambda x: Benefit_18a.get_OON_text(x), axis=1)
df_medical_benefits_18a['Medicare.gov URL'] = df_medical_benefits_18a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18a = explode_inn_oon_costshare(df_medical_benefits_18a)
df_medical_benefits_18a = drop_pbp_mrx_columns(df_medical_benefits_18a)
write_pd_to_csv(df_medical_benefits_18a,  'MedicalBenefits_18a')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18a, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing exam
# Benefit Code = 18a1
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b18a_bendesc_yn,
    pbp_b18a_bendesc_ehc,
    pbp_b18a_maxenr_yn,
    pbp_b18a_maxenr_amt,
    pbp_b18a_coins_yn,    
    pbp_b18a_coins_ehc,
    pbp_b18a_med_coins_pct,
    pbp_b18a_med_coins_pct_max,
    pbp_b18a_coins_pct_rht,
    pbp_b18a_coins_pct_max_rht,
    pbp_b18a_coins_pct_fha,
    pbp_b18a_coins_pct_max_fha,
    pbp_b18a_copay_yn,
    pbp_b18a_copay_ehc,
    pbp_b18a_copay_amt,
    pbp_b18a_med_copay_amt_max,
    pbp_b18a_copay_amt_rht,
    pbp_b18a_copay_amt_max_rht,
    pbp_b18a_copay_amt_fha,
    pbp_b18a_copay_amt_max_fha,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18a;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18a1 = spark.sql(query)
write_to_csv_file(df_18a1, 'MedicalBenefits_18a1_DataSource')

df_medical_benefits_18a1 = read_pd_from_csv_file('MedicalBenefits_18a1_DataSource')
df_medical_benefits_18a1['PlanYear'] = 2024
df_medical_benefits_18a1['QID'] = df_medical_benefits_18a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18a1['CategoryName'] = 'HEARING'
df_medical_benefits_18a1['ServiceName'] = 'Hearing exam'

from PBP_Benefit_Text import Benefit_18a1
df_medical_benefits_18a1['INN_CostShare'] = df_medical_benefits_18a1.apply(lambda x: Benefit_18a1.get_INN_text(x), axis=1)
df_medical_benefits_18a1['OON_CostShare'] = df_medical_benefits_18a1.apply(lambda x: Benefit_18a1.get_OON_text(x), axis=1)
df_medical_benefits_18a1['Medicare.gov URL'] = df_medical_benefits_18a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18a1 = explode_inn_oon_costshare(df_medical_benefits_18a1)
df_medical_benefits_18a1 = drop_pbp_mrx_columns(df_medical_benefits_18a1)
write_pd_to_csv(df_medical_benefits_18a1,  'MedicalBenefits_18a1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18a1, pandas_df_medicalbenefits)  

In [ ]:
# HEARING	Fitting/evaluation
# Benefit Code = 18a2
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b18a_bendesc_yn,
    pbp_b18a_bendesc_ehc,
    pbp_b18a_maxenr_yn,
    pbp_b18a_maxenr_amt,
    pbp_b18a_coins_yn,    
    pbp_b18a_coins_ehc,
    pbp_b18a_med_coins_pct,
    pbp_b18a_med_coins_pct_max,
    pbp_b18a_coins_pct_rht,
    pbp_b18a_coins_pct_max_rht,
    pbp_b18a_coins_pct_fha,
    pbp_b18a_coins_pct_max_fha,
    pbp_b18a_copay_yn,
    pbp_b18a_copay_ehc,
    pbp_b18a_copay_amt,
    pbp_b18a_med_copay_amt_max,
    pbp_b18a_copay_amt_rht,
    pbp_b18a_copay_amt_max_rht,
    pbp_b18a_copay_amt_fha,
    pbp_b18a_copay_amt_max_fha,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18a;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18a2 = spark.sql(query)
write_to_csv_file(df_18a2, 'MedicalBenefits_18a2_DataSource')

df_medical_benefits_18a2 = read_pd_from_csv_file('MedicalBenefits_18a2_DataSource')
df_medical_benefits_18a2['PlanYear'] = 2024
df_medical_benefits_18a2['QID'] = df_medical_benefits_18a2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18a2['CategoryName'] = 'HEARING'
df_medical_benefits_18a2['ServiceName'] = 'Fitting/evaluation'

from PBP_Benefit_Text import Benefit_18a2
df_medical_benefits_18a2['INN_CostShare'] = df_medical_benefits_18a2.apply(lambda x: Benefit_18a2.get_INN_text(x), axis=1)
df_medical_benefits_18a2['OON_CostShare'] = df_medical_benefits_18a2.apply(lambda x: Benefit_18a2.get_OON_text(x), axis=1)
df_medical_benefits_18a2['Medicare.gov URL'] = df_medical_benefits_18a2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18a2 = explode_inn_oon_costshare(df_medical_benefits_18a2)
df_medical_benefits_18a2 = drop_pbp_mrx_columns(df_medical_benefits_18a2)
write_pd_to_csv(df_medical_benefits_18a2,  'MedicalBenefits_18a2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18a2, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing aids - all types
# Benefit Code = 18b1
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,    
    pbp_b18b_bendesc_yn,
    pbp_b18b_bendesc_ehc,
    pbp_b18b_maxenr_yn,
    pbp_b18b_maxenr_amt,
    pbp_b18b_coins_yn,
    pbp_b18b_coins_ehc, 
    pbp_b18b_coins_pct_at_min,
    pbp_b18b_coins_pct_at_max,
    pbp_b18b_coins_pct_ie_min,
    pbp_b18b_coins_pct_ie_max,
    pbp_b18b_coins_pct_oe_min,
    pbp_b18b_coins_pct_oe_max,
    pbp_b18b_coins_pct_ote_min,
    pbp_b18b_coins_pct_ote_max,
    pbp_b18b_copay_yn,
    pbp_b18b_copay_ehc,
    pbp_b18b_copay_at_min_amt,
    pbp_b18b_copay_at_max_amt,
    pbp_b18b_copay_amt_per_ie_min,
    pbp_b18b_copay_amt_per_ie_max,
    pbp_b18b_copay_amt_p2_ie_min,
    pbp_b18b_copay_amt_p2_ie_max,
    pbp_b18b_copay_amt_per_oe_min,
    pbp_b18b_copay_amt_per_oe_max,
    pbp_b18b_copay_amt_p2_oe_min,
    pbp_b18b_copay_amt_p2_oe_max,
    pbp_b18b_copay_amt_per_ote_min,
    pbp_b18b_copay_amt_per_ote_max,
    pbp_b18b_copay_amt_p2_ote_min,
    pbp_b18b_copay_amt_p2_ote_max,   
    
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids bmc on c.bid_id = bmc.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18b = spark.sql(query)
write_to_csv_file(df_18b, 'MedicalBenefits_18b_DataSource')
df_medical_benefits_18b1 = read_pd_from_csv_file('MedicalBenefits_18b_DataSource')
df_medical_benefits_18b1['PlanYear'] = 2024
df_medical_benefits_18b1['QID'] = df_medical_benefits_18b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18b1['CategoryName'] = 'HEARING'
df_medical_benefits_18b1['ServiceName'] = 'Hearing aids - all types'

from PBP_Benefit_Text import Benefit_18b1
df_medical_benefits_18b1['INN_CostShare'] = df_medical_benefits_18b1.apply(lambda x: Benefit_18b1.get_INN_text(x), axis=1)
df_medical_benefits_18b1['OON_CostShare'] = df_medical_benefits_18b1.apply(lambda x: Benefit_18b1.get_OON_text(x), axis=1)
df_medical_benefits_18b1['Medicare.gov URL'] = df_medical_benefits_18b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18b1 = explode_inn_oon_costshare(df_medical_benefits_18b1)
df_medical_benefits_18b1 = drop_pbp_mrx_columns(df_medical_benefits_18b1)
write_pd_to_csv(df_medical_benefits_18b1,  'MedicalBenefits_18b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18b1, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing aids - Inner ear
# Benefit Code = 18b2
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,    
    pbp_b18b_bendesc_yn,
    pbp_b18b_bendesc_ehc,
    pbp_b18b_maxenr_yn,
    pbp_b18b_maxenr_amt,
    pbp_b18b_coins_yn,
    pbp_b18b_coins_ehc, 
    pbp_b18b_coins_pct_at_min,
    pbp_b18b_coins_pct_at_max,
    pbp_b18b_coins_pct_ie_min,
    pbp_b18b_coins_pct_ie_max,
    pbp_b18b_coins_pct_oe_min,
    pbp_b18b_coins_pct_oe_max,
    pbp_b18b_coins_pct_ote_min,
    pbp_b18b_coins_pct_ote_max,
    pbp_b18b_copay_yn,
    pbp_b18b_copay_ehc,
    pbp_b18b_copay_at_min_amt,
    pbp_b18b_copay_at_max_amt,
    pbp_b18b_copay_amt_per_ie_min,
    pbp_b18b_copay_amt_per_ie_max,
    pbp_b18b_copay_amt_p2_ie_min,
    pbp_b18b_copay_amt_p2_ie_max,
    pbp_b18b_copay_amt_per_oe_min,
    pbp_b18b_copay_amt_per_oe_max,
    pbp_b18b_copay_amt_p2_oe_min,
    pbp_b18b_copay_amt_p2_oe_max,
    pbp_b18b_copay_amt_per_ote_min,
    pbp_b18b_copay_amt_per_ote_max,
    pbp_b18b_copay_amt_p2_ote_min,
    pbp_b18b_copay_amt_p2_ote_max,   
    pbp_b18b_bendesc_lim_ie,
    pbp_b18b_bendesc_numv_ie,
    pbp_b18b_bendesc_per_ie,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids bmc on c.bid_id = bmc.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18b = spark.sql(query)
write_to_csv_file(df_18b, 'MedicalBenefits_18b_DataSource')
df_medical_benefits_18b2 = read_pd_from_csv_file('MedicalBenefits_18b_DataSource')
df_medical_benefits_18b2['PlanYear'] = 2024
from PBP_Benefit_Text import Benefit_18b2, Plan
df_medical_benefits_18b2['QID'] = df_medical_benefits_18b2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18b2['CategoryName'] = 'HEARING'
df_medical_benefits_18b2['ServiceName'] = 'Hearing aids - Inner ear'

df_medical_benefits_18b2['INN_CostShare'] = df_medical_benefits_18b2.apply(lambda x: Benefit_18b2.get_INN_text(x), axis=1)
df_medical_benefits_18b2['OON_CostShare'] = df_medical_benefits_18b2.apply(lambda x: Benefit_18b2.get_OON_text(x), axis=1)
df_medical_benefits_18b2['Medicare.gov URL'] = df_medical_benefits_18b2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18b2 = explode_inn_oon_costshare(df_medical_benefits_18b2)
df_medical_benefits_18b2 = drop_pbp_mrx_columns(df_medical_benefits_18b2)
write_pd_to_csv(df_medical_benefits_18b2,  'MedicalBenefits_18b2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18b2, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing aids - Outer ear
# Benefit Code = 18b3
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,    
    pbp_b18b_bendesc_yn,
    pbp_b18b_bendesc_ehc,
    pbp_b18b_maxenr_yn,
    pbp_b18b_maxenr_amt,
    pbp_b18b_coins_yn,
    pbp_b18b_coins_ehc, 
    pbp_b18b_coins_pct_at_min,
    pbp_b18b_coins_pct_at_max,
    pbp_b18b_coins_pct_ie_min,
    pbp_b18b_coins_pct_ie_max,
    pbp_b18b_coins_pct_oe_min,
    pbp_b18b_coins_pct_oe_max,
    pbp_b18b_coins_pct_ote_min,
    pbp_b18b_coins_pct_ote_max,
    pbp_b18b_copay_yn,
    pbp_b18b_copay_ehc,
    pbp_b18b_copay_at_min_amt,
    pbp_b18b_copay_at_max_amt,
    pbp_b18b_copay_amt_per_ie_min,
    pbp_b18b_copay_amt_per_ie_max,
    pbp_b18b_copay_amt_p2_ie_min,
    pbp_b18b_copay_amt_p2_ie_max,
    pbp_b18b_copay_amt_per_oe_min,
    pbp_b18b_copay_amt_per_oe_max,
    pbp_b18b_copay_amt_p2_oe_min,
    pbp_b18b_copay_amt_p2_oe_max,
    pbp_b18b_copay_amt_per_ote_min,
    pbp_b18b_copay_amt_per_ote_max,
    pbp_b18b_copay_amt_p2_ote_min,
    pbp_b18b_copay_amt_p2_ote_max,   
    pbp_b18b_bendesc_lim_ie,
    pbp_b18b_bendesc_numv_ie,
    pbp_b18b_bendesc_per_ie,
    pbp_b18b_bendesc_lim_oe,
    pbp_b18b_bendesc_numv_oe,
    pbp_b18b_bendesc_per_oe,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids bmc on c.bid_id = bmc.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18b = spark.sql(query)
write_to_csv_file(df_18b, 'MedicalBenefits_18b_DataSource')
df_medical_benefits_18b3 = read_pd_from_csv_file('MedicalBenefits_18b_DataSource')
df_medical_benefits_18b3['PlanYear'] = 2024
from PBP_Benefit_Text import Benefit_18b3, Plan
df_medical_benefits_18b3['QID'] = df_medical_benefits_18b3.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18b3['CategoryName'] = 'HEARING'
df_medical_benefits_18b3['ServiceName'] = 'Hearing aids - Outer ear'

df_medical_benefits_18b3['INN_CostShare'] = df_medical_benefits_18b3.apply(lambda x: Benefit_18b3.get_INN_text(x), axis=1)
df_medical_benefits_18b3['OON_CostShare'] = df_medical_benefits_18b3.apply(lambda x: Benefit_18b3.get_OON_text(x), axis=1)
df_medical_benefits_18b3['Medicare.gov URL'] = df_medical_benefits_18b3.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18b3 = explode_inn_oon_costshare(df_medical_benefits_18b3)
df_medical_benefits_18b3 = drop_pbp_mrx_columns(df_medical_benefits_18b3)
write_pd_to_csv(df_medical_benefits_18b3,  'MedicalBenefits_18b3')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18b3, pandas_df_medicalbenefits)   

In [ ]:
# HEARING	Hearing aids - Over the ear
# Benefit Code = 18b4
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,    
    pbp_b18b_bendesc_yn,
    pbp_b18b_bendesc_ehc,
    pbp_b18b_maxenr_yn,
    pbp_b18b_maxenr_amt,
    pbp_b18b_coins_yn,
    pbp_b18b_coins_ehc, 
    pbp_b18b_coins_pct_at_min,
    pbp_b18b_coins_pct_at_max,
    pbp_b18b_coins_pct_ie_min,
    pbp_b18b_coins_pct_ie_max,
    pbp_b18b_coins_pct_oe_min,
    pbp_b18b_coins_pct_oe_max,
    pbp_b18b_coins_pct_ote_min,
    pbp_b18b_coins_pct_ote_max,
    pbp_b18b_copay_yn,
    pbp_b18b_copay_ehc,
    pbp_b18b_copay_at_min_amt,
    pbp_b18b_copay_at_max_amt,
    pbp_b18b_copay_amt_per_ie_min,
    pbp_b18b_copay_amt_per_ie_max,
    pbp_b18b_copay_amt_p2_ie_min,
    pbp_b18b_copay_amt_p2_ie_max,
    pbp_b18b_copay_amt_per_oe_min,
    pbp_b18b_copay_amt_per_oe_max,
    pbp_b18b_copay_amt_p2_oe_min,
    pbp_b18b_copay_amt_p2_oe_max,
    pbp_b18b_copay_amt_per_ote_min,
    pbp_b18b_copay_amt_per_ote_max,
    pbp_b18b_copay_amt_p2_ote_min,
    pbp_b18b_copay_amt_p2_ote_max,   
    pbp_b18b_bendesc_lim_ie,
    pbp_b18b_bendesc_numv_ie,
    pbp_b18b_bendesc_per_ie,
    pbp_b18b_bendesc_lim_oe,
    pbp_b18b_bendesc_numv_oe,
    pbp_b18b_bendesc_per_oe,
    pbp_b18b_bendesc_lim_ote,
    pbp_b18b_bendesc_numv_ote,
    pbp_b18b_bendesc_per_ote,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b18_hearing_exams_aids bmc on c.bid_id = bmc.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;18b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;18b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_18b = spark.sql(query)
write_to_csv_file(df_18b, 'MedicalBenefits_18b_DataSource')
df_medical_benefits_18b4 = read_pd_from_csv_file('MedicalBenefits_18b_DataSource')
df_medical_benefits_18b4['PlanYear'] = 2024
from PBP_Benefit_Text import Benefit_18b4, Plan
df_medical_benefits_18b4['QID'] = df_medical_benefits_18b4.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_18b4['CategoryName'] = 'HEARING'
df_medical_benefits_18b4['ServiceName'] = 'Hearing aids - Over the ear'

df_medical_benefits_18b4['INN_CostShare'] = df_medical_benefits_18b4.apply(lambda x: Benefit_18b4.get_INN_text(x), axis=1)
df_medical_benefits_18b4['OON_CostShare'] = df_medical_benefits_18b4.apply(lambda x: Benefit_18b4.get_OON_text(x), axis=1)
df_medical_benefits_18b4['Medicare.gov URL'] = df_medical_benefits_18b4.apply(lambda x: get_medicare_site_url(x.QID), axis=1)

df_medical_benefits_18b4 = explode_inn_oon_costshare(df_medical_benefits_18b4)
df_medical_benefits_18b4 = drop_pbp_mrx_columns(df_medical_benefits_18b4)
write_pd_to_csv(df_medical_benefits_18b4,  'MedicalBenefits_18b4')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_18b4, pandas_df_medicalbenefits)   

In [ ]:
# VISION	Eye wear (Medicare covered)
# Benefit Code = 17b
# In-Network & Out-of-Network

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_type,
    pbp_b17b_maxenr_amt,
    pbp_b17b_maxenr_per,
    pbp_b17b_maxenr_per_d,
    pbp_b17b_coins_yn,
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_mc_min,
    pbp_b17b_coins_pct_mc_max,
    pbp_b17b_ded_yn,
    pbp_b17b_ded_amt,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_mc_min,
    pbp_b17b_copay_amt_mc_max,
    pbp_b17b_auth_yn,
    pbp_b17b_refer_yn,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon on c.bid_id = coon.bid_id and concat(';', COALESCE(coon.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos on c.bid_id = cpos.bid_id and concat(';', COALESCE(cpos.pbp_c_pos_outpt_mc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b = spark.sql(query)
write_to_csv_file(df_17b, 'MedicalBenefits_17b_DataSource')
df_medical_benefits_17b = read_pd_from_csv_file('MedicalBenefits_17b_DataSource')
from PBP_Benefit_Text import Plan, Benefit_17b
df_medical_benefits_17b['PlanYear'] = 2024
df_medical_benefits_17b['QID'] = df_medical_benefits_17b.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b['CategoryName'] = 'VISION'
df_medical_benefits_17b['ServiceName'] = 'Eye wear (Medicare covered)'

df_medical_benefits_17b['INN_CostShare'] = df_medical_benefits_17b.apply(lambda x: Benefit_17b.get_INN_text(x), axis=1)
df_medical_benefits_17b['OON_CostShare'] = df_medical_benefits_17b.apply(lambda x: Benefit_17b.get_OON_text(x), axis=1)
df_medical_benefits_17b['Medicare.gov URL'] = df_medical_benefits_17b.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b = explode_inn_oon_costshare(df_medical_benefits_17b)
df_medical_benefits_17b = drop_pbp_mrx_columns(df_medical_benefits_17b)
write_pd_to_csv(df_medical_benefits_17b,  'MedicalBenefits_17b')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b, pandas_df_medicalbenefits)   

In [ ]:
# VISION	Contact lenses
# Benefit Code = 17b1
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_amt,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_coins_yn,    
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_cl_min,
    pbp_b17b_coins_pct_cl_max,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_cl_min,
    pbp_b17b_copay_amt_cl_max,
    
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_yn,pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b1 = spark.sql(query)
write_to_csv_file(df_17b1, 'MedicalBenefits_17b1_DataSource')
df_medical_benefits_17b1 = read_pd_from_csv_file('MedicalBenefits_17b1_DataSource')
df_medical_benefits_17b1['PlanYear'] = 2024
df_medical_benefits_17b1['QID'] = df_medical_benefits_17b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b1['CategoryName'] = 'VISION'
df_medical_benefits_17b1['ServiceName'] = 'Contact lenses'

from PBP_Benefit_Text import Benefit_17b1
df_medical_benefits_17b1['INN_CostShare'] = df_medical_benefits_17b1.apply(lambda x: Benefit_17b1.get_INN_text(x), axis=1)
df_medical_benefits_17b1['OON_CostShare'] = df_medical_benefits_17b1.apply(lambda x: Benefit_17b1.get_OON_text(x), axis=1)
df_medical_benefits_17b1['Medicare.gov URL'] = df_medical_benefits_17b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b1 = explode_inn_oon_costshare(df_medical_benefits_17b1)
df_medical_benefits_17b1 = drop_pbp_mrx_columns(df_medical_benefits_17b1)
write_pd_to_csv(df_medical_benefits_17b1,  'MedicalBenefits_17b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b1, pandas_df_medicalbenefits) 

In [ ]:
# VISION	Eyeglasses (frames & lenses)
# Benefit Code = 17b2
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_amt,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_coins_yn,    
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_cl_min,
    pbp_b17b_coins_pct_cl_max,
    pbp_b17b_coins_pct_egs_min,
    pbp_b17b_coins_pct_egs_max,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_cl_min,
    pbp_b17b_copay_amt_cl_max,
    pbp_b17b_copay_amt_egs_min,
    pbp_b17b_copay_amt_egs_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b2 = spark.sql(query)
write_to_csv_file(df_17b2, 'MedicalBenefits_17b2_DataSource')
df_medical_benefits_17b2 = read_pd_from_csv_file('MedicalBenefits_17b2_DataSource')
df_medical_benefits_17b2['PlanYear'] = 2024
df_medical_benefits_17b2['QID'] = df_medical_benefits_17b2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b2['CategoryName'] = 'VISION'
df_medical_benefits_17b2['ServiceName'] = 'Eyeglasses (frames & lenses)'

from PBP_Benefit_Text import Benefit_17b2
df_medical_benefits_17b2['INN_CostShare'] = df_medical_benefits_17b2.apply(lambda x: Benefit_17b2.get_INN_text(x), axis=1)
df_medical_benefits_17b2['OON_CostShare'] = df_medical_benefits_17b2.apply(lambda x: Benefit_17b2.get_OON_text(x), axis=1)
df_medical_benefits_17b2['Medicare.gov URL'] = df_medical_benefits_17b2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b2 = explode_inn_oon_costshare(df_medical_benefits_17b2)
df_medical_benefits_17b2 = drop_pbp_mrx_columns(df_medical_benefits_17b2)
write_pd_to_csv(df_medical_benefits_17b2,  'MedicalBenefits_17b2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b2, pandas_df_medicalbenefits)  

In [ ]:
# VISION	Eyeglass lenses only
# Benefit Code = 17b3
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_amt,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_coins_yn,    
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_cl_min,
    pbp_b17b_coins_pct_cl_max,
    pbp_b17b_coins_pct_egs_min,
    pbp_b17b_coins_pct_egs_max,
    pbp_b17b_coins_pct_egl_min,
    pbp_b17b_coins_pct_egl_max,
    pbp_b17b_coins_pct_egf_min,
    pbp_b17b_coins_pct_egf_max,
    pbp_b17b_coins_pct_upg_min,
    pbp_b17b_coins_pct_upg_max,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_cl_min,
    pbp_b17b_copay_amt_cl_max,
    pbp_b17b_copay_amt_egs_min,
    pbp_b17b_copay_amt_egs_max,
    pbp_b17b_copay_amt_egl_min,
    pbp_b17b_copay_amt_egl_max,
    pbp_b17b_copay_amt_egf_min,
    pbp_b17b_copay_amt_egf_max,
    pbp_b17b_copay_amt_upg_min,
    pbp_b17b_copay_amt_upg_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b3 = spark.sql(query)
write_to_csv_file(df_17b3, 'MedicalBenefits_17b3_DataSource')
df_medical_benefits_17b3 = read_pd_from_csv_file('MedicalBenefits_17b3_DataSource')
df_medical_benefits_17b3['PlanYear'] = 2024
df_medical_benefits_17b3['QID'] = df_medical_benefits_17b3.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b3['CategoryName'] = 'VISION'
df_medical_benefits_17b3['ServiceName'] = 'Eyeglass lenses only'

from PBP_Benefit_Text import Benefit_17b3
df_medical_benefits_17b3['INN_CostShare'] = df_medical_benefits_17b3.apply(lambda x: Benefit_17b3.get_INN_text(x), axis=1)
df_medical_benefits_17b3['OON_CostShare'] = df_medical_benefits_17b3.apply(lambda x: Benefit_17b3.get_OON_text(x), axis=1)
df_medical_benefits_17b3['Medicare.gov URL'] = df_medical_benefits_17b3.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b3 = explode_inn_oon_costshare(df_medical_benefits_17b3)
df_medical_benefits_17b3 = drop_pbp_mrx_columns(df_medical_benefits_17b3)
write_pd_to_csv(df_medical_benefits_17b3,  'MedicalBenefits_17b3')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b3, pandas_df_medicalbenefits)  

In [ ]:
# VISION	Eyeglass frames only
# Benefit Code = 17b4
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_amt,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_coins_yn,    
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_cl_min,
    pbp_b17b_coins_pct_cl_max,
    pbp_b17b_coins_pct_egs_min,
    pbp_b17b_coins_pct_egs_max,
    pbp_b17b_coins_pct_egl_min,
    pbp_b17b_coins_pct_egl_max,
    pbp_b17b_coins_pct_egf_min,
    pbp_b17b_coins_pct_egf_max,
    pbp_b17b_coins_pct_upg_min,
    pbp_b17b_coins_pct_upg_max,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_cl_min,
    pbp_b17b_copay_amt_cl_max,
    pbp_b17b_copay_amt_egs_min,
    pbp_b17b_copay_amt_egs_max,
    pbp_b17b_copay_amt_egl_min,
    pbp_b17b_copay_amt_egl_max,
    pbp_b17b_copay_amt_egf_min,
    pbp_b17b_copay_amt_egf_max,
    pbp_b17b_copay_amt_upg_min,
    pbp_b17b_copay_amt_upg_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt

from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b4 = spark.sql(query)
write_to_csv_file(df_17b4, 'MedicalBenefits_17b4_DataSource')
df_medical_benefits_17b4 = read_pd_from_csv_file('MedicalBenefits_17b4_DataSource')
df_medical_benefits_17b4['PlanYear'] = 2024
df_medical_benefits_17b4['QID'] = df_medical_benefits_17b4.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b4['CategoryName'] = 'VISION'
df_medical_benefits_17b4['ServiceName'] = 'Eyeglass frames only'

from PBP_Benefit_Text import Benefit_17b4
df_medical_benefits_17b4['INN_CostShare'] = df_medical_benefits_17b4.apply(lambda x: Benefit_17b4.get_INN_text(x), axis=1)
df_medical_benefits_17b4['OON_CostShare'] = df_medical_benefits_17b4.apply(lambda x: Benefit_17b4.get_OON_text(x), axis=1)
df_medical_benefits_17b4['Medicare.gov URL'] = df_medical_benefits_17b4.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b4 = explode_inn_oon_costshare(df_medical_benefits_17b4)
df_medical_benefits_17b4 = drop_pbp_mrx_columns(df_medical_benefits_17b4)
write_pd_to_csv(df_medical_benefits_17b4,  'MedicalBenefits_17b4')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b4, pandas_df_medicalbenefits)  

In [ ]:
# VISION	Upgrades
# Benefit Code = 17b5
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17b_maxenr_yn,
    pbp_b17b_maxenr_amt,
    pbp_b17b_bendesc_yn,
    pbp_b17b_bendesc_ehc,
    pbp_b17b_coins_yn,    
    pbp_b17b_coins_ehc,
    pbp_b17b_coins_pct_cl_min,
    pbp_b17b_coins_pct_cl_max,
    pbp_b17b_coins_pct_egs_min,
    pbp_b17b_coins_pct_egs_max,
    pbp_b17b_coins_pct_egl_min,
    pbp_b17b_coins_pct_egl_max,
    pbp_b17b_coins_pct_egf_min,
    pbp_b17b_coins_pct_egf_max,
    pbp_b17b_coins_pct_upg_min,
    pbp_b17b_coins_pct_upg_max,
    pbp_b17b_copay_yn,
    pbp_b17b_copay_ehc,
    pbp_b17b_copay_amt_cl_min,
    pbp_b17b_copay_amt_cl_max,
    pbp_b17b_copay_amt_egs_min,
    pbp_b17b_copay_amt_egs_max,
    pbp_b17b_copay_amt_egl_min,
    pbp_b17b_copay_amt_egl_max,
    pbp_b17b_copay_amt_egf_min,
    pbp_b17b_copay_amt_egf_max,
    pbp_b17b_copay_amt_upg_min,
    pbp_b17b_copay_amt_upg_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17b;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17b;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_17b5 = spark.sql(query)
write_to_csv_file(df_17b5, 'MedicalBenefits_17b5_DataSource')
df_medical_benefits_17b5 = read_pd_from_csv_file('MedicalBenefits_17b5_DataSource')
df_medical_benefits_17b5['PlanYear'] = 2024
df_medical_benefits_17b5['QID'] = df_medical_benefits_17b5.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17b5['CategoryName'] = 'VISION'
df_medical_benefits_17b5['ServiceName'] = 'Upgrades'

from PBP_Benefit_Text import Benefit_17b5
df_medical_benefits_17b5['INN_CostShare'] = df_medical_benefits_17b5.apply(lambda x: Benefit_17b5.get_INN_text(x), axis=1)
df_medical_benefits_17b5['OON_CostShare'] = df_medical_benefits_17b5.apply(lambda x: Benefit_17b5.get_OON_text(x), axis=1)
df_medical_benefits_17b5['Medicare.gov URL'] = df_medical_benefits_17b5.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17b5 = explode_inn_oon_costshare(df_medical_benefits_17b5)
df_medical_benefits_17b5 = drop_pbp_mrx_columns(df_medical_benefits_17b5)
write_pd_to_csv(df_medical_benefits_17b5,  'MedicalBenefits_17b5')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17b5, pandas_df_medicalbenefits)

In [ ]:
# VISION	Routine eye exam
# Benefit Code = 17a1
# In-Network & Out-of-Network
# uses Non-Medicare (NMC) for out-of-network if NMC is available

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,   
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b17a_maxenr_yn,
    pbp_b17a_maxenr_amt,
    pbp_b17a_bendesc_yn,
    pbp_b17a_bendesc_ehc,
    pbp_b17a_coins_yn,    
    pbp_b17a_coins_ehc,
    pbp_b17a_coins_pct_mc_min,
    pbp_b17a_coins_pct_mc_max,
    pbp_b17a_coins_pct_rex_min,
    pbp_b17a_coins_pct_rex_max,
    pbp_b17a_coins_pct_other_min,
    pbp_b17a_coins_pct_other_max,
    pbp_b17a_copay_yn,
    pbp_b17a_copay_ehc,
    pbp_b17a_copay_amt_mc_min,
    pbp_b17a_copay_amt_mc_max,
    pbp_b17a_copay_amt_rex_min,
    pbp_b17a_copay_amt_rex_max,
    pbp_b17a_copay_amt_other_min,
    pbp_b17a_copay_amt_other_max,
    c.pbp_c_oon_yn, pbp_c_pos_yn, 
    pbp_c_oon_outpt_maxplan_amt,pbp_c_oon_outpt_maxplan_per,pbp_c_oon_outpt_maxplan_per_d,
    pbp_c_oon_outpt_coins_yn,pbp_c_oon_outpt_coins_min_pct,pbp_c_oon_outpt_coins_max_pct,
    pbp_c_oon_outpt_copay_yn,pbp_c_oon_outpt_copay_min_amt,pbp_c_oon_outpt_copay_max_amt,
    pbp_c_oon_outpt_ded_yn,pbp_c_oon_outpt_ded_amt,
    pbp_c_pos_yn,
    pbp_c_pos_outpt_coins_yn,pbp_c_pos_outpt_coins_min_pct,pbp_c_pos_outpt_coins_max_pct,
    pbp_c_pos_outpt_copay_yn,pbp_c_pos_outpt_copay_min_amt,pbp_c_pos_outpt_copay_max_amt,
    pbp_c_pos_outpt_maxplan_yn,pbp_c_pos_outpt_maxplan_amt,pbp_c_pos_outpt_maxplan_per,pbp_c_pos_outpt_maxplan_per_d,pbp_c_pos_outpt_deduct_yn,pbp_c_pos_outpt_deduct_amt
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b17_eye_exams_wear b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_nmc on c.bid_id = coon_nmc.bid_id and concat(';', COALESCE(coon_nmc.pbp_c_oon_out_nmc_bendesc_cats, '')) like '%;17a;%'
left join pbp_Section_C_POS cpos_nmc on c.bid_id = cpos_nmc.bid_id and concat(';', COALESCE(cpos_nmc.pbp_c_pos_outpt_nmc_bencats, '')) like '%;17a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

df_17a1 = spark.sql(query)
write_to_csv_file(df_17a1, 'MedicalBenefits_17a1_DataSource')
df_medical_benefits_17a1 = read_pd_from_csv_file('MedicalBenefits_17a1_DataSource')
df_medical_benefits_17a1['PlanYear'] = 2024
df_medical_benefits_17a1['QID'] = df_medical_benefits_17a1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_17a1['CategoryName'] = 'VISION'
df_medical_benefits_17a1['ServiceName'] = 'Routine eye exam'

from PBP_Benefit_Text import Benefit_17a1
df_medical_benefits_17a1['INN_CostShare'] = df_medical_benefits_17a1.apply(lambda x: Benefit_17a1.get_INN_text(x), axis=1)
df_medical_benefits_17a1['OON_CostShare'] = df_medical_benefits_17a1.apply(lambda x: Benefit_17a1.get_OON_text(x), axis=1)
df_medical_benefits_17a1['Medicare.gov URL'] = df_medical_benefits_17a1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_17a1 = explode_inn_oon_costshare(df_medical_benefits_17a1)
df_medical_benefits_17a1 = drop_pbp_mrx_columns(df_medical_benefits_17a1)
write_pd_to_csv(df_medical_benefits_17a1,  'MedicalBenefits_17a1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_17a1, pandas_df_medicalbenefits)

In [ ]:
# PART B DRUGS	Chemotherapy drugs
# Benefit Code = 15-2
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    mrx_b_max_oop_yn,mrx_b_max_oop_amt,
mrx_b_coins_yn,
mrx_b_coins_ehc,
mrx_b_chemo_coins_min_pct,
mrx_b_chemo_coins_max_pct,
mrx_b_coins_min_pct,
mrx_b_coins_max_pct,
mrx_b_copay_yn,
mrx_b_copay_ehc,
mrx_b_chemo_copay_amt_min,
mrx_b_chemo_copay_amt_max,
mrx_b_copay_min_amt,
mrx_b_copay_max_amt,
mrx_b_ira_coins_yn,
mrx_b_ira_coins_min_pct,
mrx_b_ira_coins_max_pct,
mrx_b_ira_copay_month_amt,
mrx_b_ira_copay_yn,
mrx_b_ira_copay_amt_min,
mrx_b_ira_copay_amt_max,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b15_partb_rx_drugs b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;15;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;15;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

df_15_2 = spark.sql(query)
write_to_csv_file(df_15_2, 'MedicalBenefits_15_2_DataSource')
df_medical_benefits_15_2 = read_pd_from_csv_file('MedicalBenefits_15_2_DataSource')
df_medical_benefits_15_2['PlanYear'] = 2024
df_medical_benefits_15_2['QID'] = df_medical_benefits_15_2.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_15_2['CategoryName'] = 'PART B DRUGS'
df_medical_benefits_15_2['ServiceName'] = 'Chemotherapy drugs'

from PBP_Benefit_Text import Benefit_15_2
df_medical_benefits_15_2['INN_CostShare'] = df_medical_benefits_15_2.apply(lambda x: Benefit_15_2.get_INN_text(x), axis=1)
df_medical_benefits_15_2['OON_CostShare'] = df_medical_benefits_15_2.apply(lambda x: Benefit_15_2.get_OON_text(x), axis=1)
df_medical_benefits_15_2['Medicare.gov URL'] = df_medical_benefits_15_2.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_15_2 = explode_inn_oon_costshare(df_medical_benefits_15_2)
df_medical_benefits_15_2 = drop_pbp_mrx_columns(df_medical_benefits_15_2)
write_pd_to_csv(df_medical_benefits_15_2,  'MedicalBenefits_15_2')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_15_2, pandas_df_medicalbenefits)

In [ ]:
# PART B DRUGS	Other Part B drugs
# Benefit Code = 15-3
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    mrx_b_max_oop_yn,mrx_b_max_oop_amt,
mrx_b_coins_yn,
mrx_b_coins_ehc,
mrx_b_chemo_coins_min_pct,
mrx_b_chemo_coins_max_pct,
mrx_b_coins_min_pct,
mrx_b_coins_max_pct,
mrx_b_copay_yn,
mrx_b_copay_ehc,
mrx_b_chemo_copay_amt_min,
mrx_b_chemo_copay_amt_max,
mrx_b_copay_min_amt,
mrx_b_copay_max_amt,
mrx_b_ira_coins_yn,
mrx_b_ira_coins_min_pct,
mrx_b_ira_coins_max_pct,
mrx_b_ira_copay_month_amt,
mrx_b_ira_copay_yn,
mrx_b_ira_copay_amt_min,
mrx_b_ira_copay_amt_max,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b15_partb_rx_drugs b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;15;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;15;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

df_15_3 = spark.sql(query)
write_to_csv_file(df_15_3, 'MedicalBenefits_15_3_DataSource')
df_medical_benefits_15_3 = read_pd_from_csv_file('MedicalBenefits_15_3_DataSource')
df_medical_benefits_15_3['PlanYear'] = 2024
df_medical_benefits_15_3['QID'] = df_medical_benefits_15_3.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_15_3['CategoryName'] = 'PART B DRUGS'
df_medical_benefits_15_3['ServiceName'] = 'Other Part B drugs'

from PBP_Benefit_Text import Benefit_15_3
df_medical_benefits_15_3['INN_CostShare'] = df_medical_benefits_15_3.apply(lambda x: Benefit_15_3.get_INN_text(x), axis=1)
df_medical_benefits_15_3['OON_CostShare'] = df_medical_benefits_15_3.apply(lambda x: Benefit_15_3.get_OON_text(x), axis=1)
df_medical_benefits_15_3['Medicare.gov URL'] = df_medical_benefits_15_3.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_15_3 = explode_inn_oon_costshare(df_medical_benefits_15_3)
df_medical_benefits_15_3 = drop_pbp_mrx_columns(df_medical_benefits_15_3)
write_pd_to_csv(df_medical_benefits_15_3,  'MedicalBenefits_15_3')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_15_3, pandas_df_medicalbenefits)

In [ ]:
# PART B DRUGS	Part B insulin
# Benefit Code = 15-1-I
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,  
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    mrx_b_max_oop_yn,mrx_b_max_oop_amt,
mrx_b_coins_yn,
mrx_b_coins_ehc,
mrx_b_chemo_coins_min_pct,
mrx_b_chemo_coins_max_pct,
mrx_b_coins_min_pct,
mrx_b_coins_max_pct,
mrx_b_copay_yn,
mrx_b_copay_ehc,
mrx_b_chemo_copay_amt_min,
mrx_b_chemo_copay_amt_max,
mrx_b_copay_min_amt,
mrx_b_copay_max_amt,
mrx_b_ira_coins_yn,
mrx_b_ira_coins_min_pct,
mrx_b_ira_coins_max_pct,
mrx_b_ira_copay_month_amt,
mrx_b_ira_copay_yn,
mrx_b_ira_copay_amt_min,
mrx_b_ira_copay_amt_max,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b15_partb_rx_drugs b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;15;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;15;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''

df_15_1_I = spark.sql(query)
write_to_csv_file(df_15_1_I, 'MedicalBenefits_15_1_I_DataSource')
df_medical_benefits_15_1_I = read_pd_from_csv_file('MedicalBenefits_15_1_I_DataSource')
df_medical_benefits_15_1_I['PlanYear'] = 2024
df_medical_benefits_15_1_I['QID'] = df_medical_benefits_15_1_I.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_15_1_I['CategoryName'] = 'PART B DRUGS'
df_medical_benefits_15_1_I['ServiceName'] = 'Part B insulin'

from PBP_Benefit_Text import Benefit_15_1_I
df_medical_benefits_15_1_I['INN_CostShare'] = df_medical_benefits_15_1_I.apply(lambda x: Benefit_15_1_I.get_INN_text(x), axis=1)
df_medical_benefits_15_1_I['OON_CostShare'] = df_medical_benefits_15_1_I.apply(lambda x: Benefit_15_1_I.get_OON_text(x), axis=1)
df_medical_benefits_15_1_I['Medicare.gov URL'] = df_medical_benefits_15_1_I.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_15_1_I = explode_inn_oon_costshare(df_medical_benefits_15_1_I)
df_medical_benefits_15_1_I = drop_pbp_mrx_columns(df_medical_benefits_15_1_I)
write_pd_to_csv(df_medical_benefits_15_1_I,  'MedicalBenefits_15_1_I')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_15_1_I, pandas_df_medicalbenefits)

In [ ]:
# OTHER SERVICES    Durable medical equipment (like wheelchairs & oxygen)
# Benefit Code = 11a
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,   
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b11a_coins_yn,
    pbp_b11a_coins_pct_mc,
    pbp_b11a_coins_pct_mcmax,
    pbp_b11a_copay_yn,
    pbp_b11a_copay_mc_amt,
    pbp_b11a_copay_mcmax_amt,
    pbp_b11a_mm_coins_yn,
    pbp_b11a_mm_coins_services,
    pbp_b11a_mm_coins_dme_min,
    pbp_b11a_mm_coins_dme_max,
    pbp_b11a_mm_copay_yn,
    pbp_b11a_mm_copay_services,
    pbp_b11a_mm_copay_dme_min,
    pbp_b11a_mm_copay_dme_max,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
from 
pbp_Section_A a 
inner join pbp_Section_C c on a.bid_id = c.bid_id
inner join pbp_b11_dme_prosth_orth_sup b on c.bid_id = b.bid_id
left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;11a;%'
left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;11a;%'
where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_11a = spark.sql(query)
write_to_csv_file(df_11a, 'MedicalBenefits_11a_DataSource')
df_medical_benefits_11a = read_pd_from_csv_file('MedicalBenefits_11a_DataSource')
df_medical_benefits_11a['PlanYear'] = 2024
df_medical_benefits_11a['QID'] = df_medical_benefits_11a.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_11a['CategoryName'] = 'OTHER SERVICES'
df_medical_benefits_11a['ServiceName'] = 'Durable medical equipment (like wheelchairs & oxygen)'

from PBP_Benefit_Text import Benefit_11a
df_medical_benefits_11a['INN_CostShare'] = df_medical_benefits_11a.apply(lambda x: Benefit_11a.get_INN_text(x), axis=1)
df_medical_benefits_11a['OON_CostShare'] = df_medical_benefits_11a.apply(lambda x: Benefit_11a.get_OON_text(x), axis=1)
df_medical_benefits_11a['Medicare.gov URL'] = df_medical_benefits_11a.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_11a = explode_inn_oon_costshare(df_medical_benefits_11a)
df_medical_benefits_11a = drop_pbp_mrx_columns(df_medical_benefits_11a)
write_pd_to_csv(df_medical_benefits_11a,  'MedicalBenefits_11a')


In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_11a, pandas_df_medicalbenefits)

In [ ]:
# OTHER SERVICES    Prosthetics (like braces, artificial limbs)
# Benefit Code = 11b1
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,    
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b11b_coins_yn,
    pbp_b11b_coins_ehc,
    pbp_b11b_coins_pct_mc,
    pbp_b11b_coins_pct_mcmax,
    pbp_b11b_coins_pct_mcms_min,
    pbp_b11b_coins_pct_mcms_max,
    pbp_b11b_copay_yn,
    pbp_b11b_copay_ehc,
    pbp_b11b_copay_mcmin_amt,
    pbp_b11b_copay_mcmax_amt,
    pbp_b11b_copay_mcms_min_amt,
    pbp_b11b_copay_mcms_max_amt,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
    from 
    pbp_Section_A a 
    inner join pbp_Section_C c on a.bid_id = c.bid_id
    inner join pbp_b11_dme_prosth_orth_sup b on c.bid_id = b.bid_id
    left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;11b;%'
    left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;11b;%'
    where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
    and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_11b1 = spark.sql(query)
write_to_csv_file(df_11b1, 'MedicalBenefits_11b1_DataSource')
df_medical_benefits_11b1 = read_pd_from_csv_file('MedicalBenefits_11b1_DataSource')
df_medical_benefits_11b1['PlanYear'] = 2024
df_medical_benefits_11b1['QID'] = df_medical_benefits_11b1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_11b1['CategoryName'] = 'OTHER SERVICES'
df_medical_benefits_11b1['ServiceName'] = 'Prosthetics (like braces, artificial limbs)'

from PBP_Benefit_Text import Benefit_11b1
df_medical_benefits_11b1['INN_CostShare'] = df_medical_benefits_11b1.apply(lambda x: Benefit_11b1.get_INN_text(x), axis=1)
df_medical_benefits_11b1['OON_CostShare'] = df_medical_benefits_11b1.apply(lambda x: Benefit_11b1.get_OON_text(x), axis=1)
df_medical_benefits_11b1['Medicare.gov URL'] = df_medical_benefits_11b1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_11b1 = explode_inn_oon_costshare(df_medical_benefits_11b1)
df_medical_benefits_11b1 = drop_pbp_mrx_columns(df_medical_benefits_11b1)
write_pd_to_csv(df_medical_benefits_11b1,  'MedicalBenefits_11b1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_11b1, pandas_df_medicalbenefits)

In [ ]:
# OTHER SERVICES   Diabetes supplies
# Benefit Code = 11c1
# In-Network & Out-of-Network
# uses Medicare (MC) for out-of-network 

query = f'''select	
	a.PBP_A_CONTRACT_NUMBER as ContractID,	
	a.pbp_a_plan_identifier1 as PlanID, 	
	a.PBP_A_SEGMENT_ID as SegmentID,    
    pbp_a_special_need_plan_type, pbp_a_dsnp_zerodollar, pbp_a_snp_state_cvg_yn,
    pbp_b11c_coins_yn,
    pbp_b11c_coins_ehc,
pbp_b11c_coins_pct_mcmin,
pbp_b11c_coins_pct_mcmax,
pbp_b11c_coins_dtsi_mcmin_pct,
pbp_b11c_coins_dtsi_mcmax_pct,
pbp_b11c_copay_yn,
pbp_b11c_copay_ehc,
pbp_b11c_copay_mcmin_amt,
pbp_b11c_copay_mcmax_amt,
pbp_b11c_copay_dtsi_mcmin_amt,
pbp_b11c_copay_dtsi_mcmax_amt,
    pbp_c_oon_yn, pbp_c_pos_yn,
pbp_c_oon_outpt_maxplan_yn,
pbp_c_oon_outpt_maxplan_amt,
pbp_c_oon_outpt_maxplan_per,
pbp_c_oon_outpt_maxplan_per_d,
pbp_c_oon_outpt_coins_yn,
pbp_c_oon_outpt_coins_min_pct,
pbp_c_oon_outpt_coins_max_pct,
pbp_c_oon_outpt_copay_yn,
pbp_c_oon_outpt_copay_min_amt,
pbp_c_oon_outpt_copay_max_amt,
pbp_c_oon_outpt_ded_yn,
pbp_c_oon_outpt_ded_amt,
pbp_c_pos_outpt_coins_yn,
pbp_c_pos_outpt_coins_min_pct,
pbp_c_pos_outpt_coins_max_pct,
pbp_c_pos_outpt_copay_yn,
pbp_c_pos_outpt_copay_min_amt,
pbp_c_pos_outpt_copay_max_amt,
pbp_c_pos_outpt_maxplan_yn,
pbp_c_pos_outpt_maxplan_amt,
pbp_c_pos_outpt_maxplan_per,
pbp_c_pos_outpt_maxplan_per_d,
pbp_c_pos_outpt_deduct_yn
    from 
    pbp_Section_A a 
    inner join pbp_Section_C c on a.bid_id = c.bid_id
    inner join pbp_b11_dme_prosth_orth_sup b on c.bid_id = b.bid_id
    left join pbp_Section_C_OON coon_mc on c.bid_id = coon_mc.bid_id and concat(';', COALESCE(coon_mc.pbp_c_oon_out_mc_bendesc_cats, '')) like '%;11c;%'
    left join pbp_Section_C_POS cpos_mc on c.bid_id = cpos_mc.bid_id and concat(';', COALESCE(cpos_mc.pbp_c_pos_outpt_mc_bencats, '')) like '%;11c;%'
    where CAST(c.pbp_a_plan_identifier AS INT) < 800 and cast(pbp_a_eghp_yn as int) = 2  
    and cast(c.PBP_A_PLAN_TYPE as int) in (1, 2, 4, 9, 29, 31, 42, 43, 44, 45 )
'''
df_11c1 = spark.sql(query)
write_to_csv_file(df_11c1, 'MedicalBenefits_11c1_DataSource')
df_medical_benefits_11c1 = read_pd_from_csv_file('MedicalBenefits_11c1_DataSource')
df_medical_benefits_11c1['PlanYear'] = 2024
df_medical_benefits_11c1['QID'] = df_medical_benefits_11c1.apply(lambda x: Plan.get_QID(x), axis=1)
df_medical_benefits_11c1['CategoryName'] = 'OTHER SERVICES'
df_medical_benefits_11c1['ServiceName'] = 'Diabetes supplies'

from PBP_Benefit_Text import Benefit_11c1
df_medical_benefits_11c1['INN_CostShare'] = df_medical_benefits_11c1.apply(lambda x: Benefit_11c1.get_INN_text(x), axis=1)
df_medical_benefits_11c1['OON_CostShare'] = df_medical_benefits_11c1.apply(lambda x: Benefit_11c1.get_OON_text(x), axis=1)
df_medical_benefits_11c1['Medicare.gov URL'] = df_medical_benefits_11c1.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
df_medical_benefits_11c1 = explode_inn_oon_costshare(df_medical_benefits_11c1)
df_medical_benefits_11c1 = drop_pbp_mrx_columns(df_medical_benefits_11c1)
write_pd_to_csv(df_medical_benefits_11c1,  'MedicalBenefits_11c1')

In [ ]:
pandas_df_medicalbenefits = add_category_benefit_to_df_medicalbenefits(df_medical_benefits_11c1, pandas_df_medicalbenefits)

In [ ]:
write_pd_to_csv(pandas_df_medicalbenefits, 'MedicareBenefits')

In [ ]:
if len(df_plans) > 0 and len(pandas_df_medicalbenefits) > 0:
    df_plan_x_benefit = pd.merge(df_plans, pandas_df_medicalbenefits, how='inner', on=['PlanYear', 'QID'])
    df_plan_x_benefit['Medicare.gov URL'] = df_plan_x_benefit.apply(lambda x: get_medicare_site_url(x.QID), axis=1)
    write_pd_to_csv(df_plan_x_benefit, f"Plan_MedicalBenefits_PBP_{date.today()}")

In [ ]:
# drop views from memory if the view exist
for t in spark.catalog.listTables():
    spark.catalog.dropTempView(t.name)